In [1]:
import pandas as pd 
import numpy as np 
import re
from functools import partial
from typing import List, Dict
import json
import os

In [132]:
recipes_df = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/df_unique_recipes_6865.csv", 
                         sep="|")

In [133]:
sum(recipes_df["calories"].isna())

2394

In [134]:
mask = recipes_df["calories"].isna()
calorie_completion = recipes_df.loc[mask, :]
print(calorie_completion.shape)

(2394, 9)


In [135]:
def extract_calories(text: str, pattern: str):
    lower_text = text.lower()
    # split the text 
    splits = lower_text.split("\n")
    for split in splits:
        result = re.findall(pattern, split)
        if len(result) == 1:
            return result[0]

In [108]:
processing_function = partial(extract_calories, 
                              pattern=r"calories per portion:|calories: (\d+)")

In [136]:
temp_series = calorie_completion['raw_text'].apply(lambda x: processing_function(x))

In [138]:
calorie_completion.loc[temp_series.index, 'calories'] = temp_series.values

In [139]:
sum(calorie_completion["calories"].isna())

599

In [140]:
second_processing = calorie_completion.loc[calorie_completion["calories"].isna(), :]

In [144]:
processing_function_v2 = partial(extract_calories, 
                              pattern=r"(\d+)\s*kcal")

In [145]:
temp_series = second_processing['raw_text'].apply(lambda x: processing_function_v2(x))

In [146]:
sum(temp_series.isna())

0

In [147]:
calorie_completion.loc[temp_series.index, 'calories'] = temp_series.values

In [149]:
sum(calorie_completion["calories"].isna())

0

In [150]:
recipes_df.loc[calorie_completion.index, "calories"] = calorie_completion.loc[:, "calories"]

In [151]:
sum(recipes_df["calories"].isna())

0

In [152]:
recipes_df["calories"]

0        70.0
1       150.0
2       200.0
3       250.0
4       300.0
        ...  
6860    450.0
6861    350.0
6862    420.0
6863    480.0
6864    350.0
Name: calories, Length: 6865, dtype: object

In [153]:
recipes_df.to_csv("reciped_fixed_calories.csv", sep="|", index=False)

In [154]:
# Fix the 
sum(recipes_df["ingredients"].isna())

153

In [156]:
def clean_titles(text:str):
    value = text.split('\n')[0]
    return value

In [164]:
recipes_df['title']=recipes_df['title'].apply(lambda x: clean_titles(x))

In [165]:
unique_recipes = recipes_df['title'].unique()

In [166]:
len(unique_recipes)

6195

In [168]:
sum(recipes_df['title'] == 'Tofu Scramble')

1

In [ ]:
# remove duplicates 

In [169]:
recipes_to_fix = recipes_df.loc[recipes_df["ingredients"].isna(), "title"].tolist()

In [173]:
with open("recipes_to_fix.json", 'w') as fp:
    json.dump(recipes_to_fix, fp)

In [175]:
recipes_df.shape

(6865, 9)

In [188]:
def check_text(text: str, word_list:List[str]):
    try:
        lower_text = text.lower()
        for word in word_list:
            if word in lower_text:
                return True
        return False
    except Exception as e:
        print(f"error in processing: {text}, {e}.")
        return False

In [189]:
non_empty_recipes = recipes_df.loc[~recipes_df["ingredients"].isna(), :]

In [190]:
# check answers 
word_list = ["apologies", "sorry", "apology"]
partial_check_text = partial(check_text, word_list=word_list)
ans = non_empty_recipes["ingredients"].apply(lambda x: partial_check_text(x))

In [191]:
sum(ans)

20

In [193]:
bad_generated_recipes = non_empty_recipes.loc[ans, :] 

In [194]:
bad_generated_recipes

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation
644,Vegan Banana Bread,12. Vegan Banana Bread\n Calories: 230 per ...,afternoon snacks,vegan,230,NaN,food_691,"sorry, i am an ai language model and i don't h...",\n1. preheat your oven to 350°f (175°c) and gr...
656,Vegan Granola Bars,24. Vegan Granola Bars\n Calories: 150 per ...,afternoon snacks,vegan,150,NaN,food_703,"i'm sorry, but as a text-based ai, i can't dir...",\n1. preheat your oven to 350°f (175°c) and li...
660,Chickpea Salad Wraps,28. Chickpea Salad Wraps\n Calories: 180 pe...,afternoon snacks,vegan,180,NaN,food_707,"i'm sorry, but as an ai text model, i don't ha...","\n1. in a medium bowl, mash the chickpeas roug..."
715,Vegan Greek Salad Skewers,83. Vegan Greek Salad Skewers\n Calories: 2...,afternoon snacks,vegan,200,NaN,food_762,"i'm sorry, as a language model ai, i don't hav...","\n1. in a bowl, combine the olive oil, lemon j..."
722,Vegan Quinoa Stuffed Bell Peppers,90. Vegan Quinoa Stuffed Bell Peppers\n Cal...,afternoon snacks,vegan,170,NaN,food_769,"i'm sorry, but as an ai language model, i can'...",\n1. preheat your oven to 375°f (190°c).\n2. c...
724,Vegan Vegetable Fritters,92. Vegan Vegetable Fritters\n Calories: 13...,afternoon snacks,vegan,130,NaN,food_771,"i'm sorry, but i'm unable to provide specific ...","\n1. in a large mixing bowl, combine grated zu..."
1251,Vegan Pho,20. Vegan Pho\nCalories: 400 per portion\nPort...,lunch,vegan,400,NaN,food_1338,"i'm sorry, but as an ai language model, i don'...","\n1. in a large pot, combine the vegetable bro..."
2242,Quinoa and Roasted Vegetable Wrap,77. Quinoa and Roasted Vegetable Wrap\nCalorie...,morning snacks,vegetarian,280,NaN,food_2492,"i'm sorry, but i can't generate the ingredient...",\n1. cook the quinoa according to package inst...
3681,Malaysian Roti Canai,29. Malaysian Roti Canai\n Calories: 400 pe...,breakfast,halal,400,NaN,food_4313,"i'm sorry, but i'm unable to provide the calor...","\n1. in a large bowl, mix together the flour a..."
4907,Lemon garlic grilled shrimp,7. Lemon garlic grilled shrimp\n- Calories: 10...,dinner,halal,100,NaN,food_5764,"i'm sorry, but as a language model ai, i don't...","\n1. in a bowl, combine the minced garlic, lem..."


In [196]:
# titles to generate again 
titles_to_check = bad_generated_recipes["title"].tolist()

In [199]:
with open("recipes_to_fix_2.json", 'w') as fp:
    json.dump(titles_to_check, fp)

## Process additional recipes 

In [200]:
# load dictionaries 
base_path = "/home/victor/Documents/Expectation_data_generation/"
dict_1 = "missing_recipes_ingredients_raw.json"
dict_2 = "missing_recipes_ingredients_raw_2.json"


In [202]:
# process dictionary 
def load_data_row(path:str):
    data = None
    with open(path, 'r') as fp:
        data = json.load(fp)
    if data is not None:
        return data
    else:
        return None

In [203]:
data_row_20 = load_data_row(os.path.join(base_path, dict_2))

In [359]:
data_raw_153 = load_data_row(os.path.join(base_path, dict_1))

In [361]:
need_to_query_again = []
for k,v in data_raw_153.items():
    if len(v) < 5:
        need_to_query_again.append(k)

In [362]:
need_to_query_again

['Vegan Gyoza']

In [366]:
len(data_raw_153)

151

In [207]:
# extract the information 


In [219]:
def extract_ingredients(text: str, 
                        pattern: str= r'ingredients:([\s\S]*?)(?=\*\*preparation steps\:)'):
    lower_text = text.lower()
    ingredients =  re.findall(pattern, 
                              lower_text, re.IGNORECASE)
    if len(ingredients) == 1:
        return ingredients[0]
    else:
        return None

In [220]:
def generate_ingredients(dict_raw_text: Dict[str, str], 
                         pattern: str = r'ingredients:([\s\S]*?)(?=\*\*preparation steps\:)'):
    answer_dict = {}
    for k, v in dict_raw_text.items():
        ingredients = extract_ingredients(v, 
                                          pattern=pattern)
        if ingredients:
            answer_dict[k] = ingredients
    return answer_dict

In [388]:
ingredients_dict_1 = generate_ingredients(data_raw_153, 
                                          pattern=r'Ingredients([\s\S]*)(?=Preparation|Instructions)')

In [389]:
len(ingredients_dict_1)

147

In [390]:
second_processing_dict = {k:v for k,v in data_raw_153.items() if k not in ingredients_dict_1.keys()}

In [392]:
len(second_processing_dict)

4

In [393]:
print(list(second_processing_dict.keys()))

['Bean Burritos', 'Tofu Stir Fry', 'Grilled Eggplant Rolls', 'Roasted Vegetable and Goat Cheese Pizza']


In [406]:
ingredients_dict_2 =  generate_ingredients(second_processing_dict, 
                                          pattern=r'Ingredient List:([\s\S]*)(?=Preparation Steps:)')

In [409]:
len(ingredients_dict_2)

4

In [410]:
ingredients_dict_1.update(ingredients_dict_2)

In [412]:
len(ingredients_dict_1)

151

In [413]:
#second_processing_dict = {k: v for k, v in second_processing_dict.items() if k not in list(ingredients_dict_5.keys())}

In [414]:
list_keys = list(second_processing_dict.keys())
print(len(list_keys))

4


In [417]:
list_keys

['Bean Burritos',
 'Tofu Stir Fry',
 'Grilled Eggplant Rolls',
 'Roasted Vegetable and Goat Cheese Pizza']

In [283]:
#ingredients_total_dict = {}
#len(ingredients_total_dict)

In [418]:
# ingredients_total_dict['Ratatouille with crusty bread'] = """
# For the Ratatouille:

# 1 large eggplant
# 2 medium zucchini
# 2 red bell peppers
# 2 yellow bell peppers
# 2 onions
# 4 cloves garlic
# 4 ripe tomatoes
# 1/4 cup olive oil
# 2 teaspoons dried thyme
# 2 teaspoons dried basil
# Salt and pepper to taste
# Fresh basil leaves for garnish (optional)
# For the Crusty Bread:

# 1 loaf of crusty bread (such as baguette)
# """

## Process the preparation steps 

In [419]:
preparation_steps_dict = {}

In [301]:
# preparation_steps_dict['Ratatouille with crusty bread'] = """
# 1. Prepare the Vegetables:

# * Wash and peel the eggplant, if desired. Cut it into 1-inch cubes.
# * Wash the zucchini and cut them into 1-inch thick slices.
# * Core and seed the bell peppers, then cut them into 1-inch strips.
# * Peel and chop the onions.
# * Peel and mince the garlic.
# * Blanch the tomatoes in boiling water for about 1-2 minutes, then transfer them to ice water to cool. This will make it easier to peel them. Remove the skins, core, and chop the tomatoes.
# 2. Sauté the Vegetables:

# * In a large, deep skillet or a Dutch oven, heat the olive oil over medium heat.
# * Add the chopped onions and minced garlic and sauté for about 2-3 minutes, until they become translucent.
# 3. Add the Vegetables:

# * Add the eggplant cubes to the skillet and cook for about 5 minutes, stirring occasionally.
# * Next, add the zucchini and bell peppers, and continue to cook for another 5 minutes.
# 4. Season and Simmer:

# * Stir in the chopped tomatoes, dried thyme, dried basil, salt, and pepper.
# * Reduce the heat to low, cover the skillet, and simmer the mixture for about 30-40 minutes, or until the vegetables are tender and the flavors have melded together. Stir occasionally.

# 5. Prepare the Crusty Bread:

# * While the Ratatouille is simmering, preheat your oven to 350°F (175°C).
# * Slice the crusty bread into thick slices and place them on a baking sheet.
# * Toast the bread in the oven for about 10-15 minutes, or until it's crusty and slightly golden.
# 6. Serve:

# * Once the Ratatouille is ready, taste and adjust the seasoning if needed.
# * Serve the Ratatouille hot, garnished with fresh basil leaves if desired, alongside the crusty bread.
# Enjoy your delicious homemade Ratatouille with crusty bread! It's a hearty and satisfying dish that's perfect for any time of the year.
# """

In [ ]:
# get the rest o the procedures 

In [425]:
list_keys = list(data_raw_153.keys())
print(len(list_keys))

151


In [421]:
def get_preparation_instructions(text:str, pattern: str):
    lower_text = text.lower()
    instructions = re.findall(pattern, lower_text, re.IGNORECASE)
    if len(instructions) == 1:
        return instructions[0]
    else:
        return None 
    

In [422]:
def get_dict_preparation_steps(raw_dict: Dict[str, str], pattern: str):
    new_dict = {}
    for k, v in raw_dict.items():
        result = get_preparation_instructions(v, pattern=pattern)
        if result:
            new_dict[k] = result
    return new_dict

In [433]:
preparation_steps_1 = get_dict_preparation_steps(data_raw_153, 
                                                 pattern='Preparation Steps([\s\S]*)')

In [434]:
len(preparation_steps_1)

147

In [435]:
second_revision = {k: v for k, v in data_raw_153.items() if k not in preparation_steps_1.keys()}

In [449]:
preparation_steps_3 = get_dict_preparation_steps(second_revision, 
                                                 pattern='Preparation([\s\S]*)')

In [450]:
preparation_steps_1.update(preparation_steps_3)

In [451]:
print(len(preparation_steps_1))

151


In [452]:
# transform in data frame 
preparation_df = pd.DataFrame(preparation_steps_1.items(), 
                              columns=["title", "preparation_steps"])

In [453]:
preparation_df.set_index("title", inplace=True)

In [455]:
# transform in data frame ingredients 
ingredients_df = pd.DataFrame(ingredients_dict_1.items(), 
                              columns=["title", 'ingredients'])

In [456]:
ingredients_df.set_index('title', inplace=True)

In [457]:
len(ingredients_dict_1)

151

In [458]:
new_recipes_df = ingredients_df.join(preparation_df)

In [459]:
new_recipes_df

,ingredients,preparation_steps
title,,
Tofu Scramble,and preparation steps:\n\ningredients:\n- 1 b...,:\n\ningredients:\n- 1 block (14 to 16 oz) of ...
Vegan Breakfast Bars,":\n- 2 cups rolled oats\n- 1/2 cup almonds, ch...",:\n1. preheat your oven to 350°f (175°c). line...
Vegan Waffles with Fruit Compote,:\n- 2 cups all-purpose flour or whole wheat f...,".\n2. in a large mixing bowl, whisk together t..."
Berry Quinoa Breakfast Bowl,:**\n\nfor the breakfast bowl:\n- 1 cup uncook...,:**\n\n1. **rinse the quinoa**: in a fine mesh...
Vegan Protein Pancakes,that are high in protein. this recipe will ma...,:\n1. **dry ingredients preparation**: in a la...
...,...,...
Stuffed Portobello mushrooms with quinoa and feta,":\n- 4 large portobello mushrooms, stems and g...",:\n\n1. preheat the oven to 375°f (190°c). lin...
Bean Burritos,\n- 1 cup of cooked and drained black beans (o...,:\n\n1. prepare the beans:\n - if you're usi...
Tofu Stir Fry,\n\n- **firm tofu:** 1 block (around 14 ounces...,:\n\n1. **press the tofu:** remove the tofu fr...


In [463]:
# load previous data frame 
recipes_20 = pd.read_csv("new_recipes_fixed_20.csv", sep='|', index_col=0)

In [466]:
fixed_recipes = pd.concat([new_recipes_df, recipes_20], axis=0)

In [470]:
fixed_recipes.shape

(171, 2)

In [477]:
text = fixed_recipes.loc["Tofu Scramble", "ingredients"]

In [488]:
a = text.split("\n\n")

In [492]:
len(a[2])

0

In [476]:
print(fixed_recipes.loc["Tofu Scramble", "ingredients"].lstrip(":"))

 and preparation steps:

ingredients:
- 1 block (14 to 16 oz) of firm or extra-firm tofu
- 1 tablespoon of olive oil or your preferred cooking oil
- 1/4 teaspoon of turmeric powder (for color)
- 1/2 teaspoon of garlic powder (or 1-2 cloves of fresh garlic, minced)
- 1/2 teaspoon of ground cumin (optional)
- 1/4 teaspoon of paprika (optional)
- salt and pepper, to taste
- 1/4 cup of nutritional yeast (for a cheesy flavor, optional)
- 1/4 cup of plant-based milk or water (if needed, for moisture)
- vegetables of choice: e.g., diced onions, bell peppers, tomatoes, spinach, kale, mushrooms (quantities as desired)
- fresh herbs like chives, parsley, or cilantro (optional, for garnish)
- squeeze of fresh lemon juice or a dash of kala namak (black salt, for an eggy flavor, optional)




In [543]:
# clean text 
def clean_text_ingredients(text: str):
    # checking integrity 
    pre_text = text
    if "ingredients" in text:
        splits = text.split("ingredients")
        pre_text = "\n".join(splits[1:])
    if pre_text.startswith(":\n") or pre_text.startswith(":**")\
        or pre_text.startswith(":") or pre_text.startswith("\n"):
        ans = pre_text.removeprefix(":\n")
        ans = ans.removeprefix("\n")
        ans = ans.removeprefix(":**")
        ans = ans.removeprefix(":")
    else:
        ans = pre_text
    if "preparation" in ans or "instructions" in ans:
        ans = ans.split("preparation")[0]
        ans = ans.split("instructions")[0]
    return ans
    # checking_list = text.split("\n\n")
    # if len(checking_list) == 1:
    #     return checking_list[0]
    # else:
    #     value_to_return = None
    #     max = -1
    #     for temp in checking_list:
    #         if len(temp) > max:
    #             value_to_return = temp
    #             max = len(temp)
    #     return value_to_return
    

In [560]:
clean_ingredients = fixed_recipes["ingredients"].apply(lambda x: clean_text_ingredients(x))

In [553]:
# clean text 
def clean_text_preparation_steps(text: str):
    # checking integrity 
    pre_text = text
    if "preparation steps:" in pre_text:
        data = pre_text.split("preparation steps:")
        if len(data) == 1:
            ans = data[0]
        else:
            ans = "\n".join(data[1:])
    elif "preparation:" in pre_text:
        data = pre_text.split("preparation:")
        if len(data) == 1:
            ans = data[0]
        else:
            ans = "\n".join(data[1:])
    elif "instructions:" in pre_text:
        data = pre_text.split("instructions:")
        if len(data) == 1:
            ans = data[0]
        else:
            ans = "\n".join(data[1:])
    else:
        ans = pre_text
    if ans.startswith(":\n") or ans.startswith(":**")\
        or ans.startswith(":") or ans.startswith("\n")\
            or ans.startswith(".\n"):
        ans = ans.removeprefix(".\n")
        ans = ans.removeprefix(":\n")
        ans = ans.removeprefix("\n")
        ans = ans.removeprefix(":**")
        ans = ans.removeprefix(":")
    return ans
    # checking_list = text.split("\n\n")
    # if len(checking_list) == 1:
    #     return checking_list[0]
    # else:
    #     value_to_return = None
    #     max = -1
    #     for temp in checking_list:
    #         if len(temp) > max:
    #             value_to_return = temp
    #             max = len(temp)
    #     return value_to_return

In [561]:
clean_steps = fixed_recipes["preparation_steps"].apply(lambda x: clean_text_preparation_steps(x))

In [569]:
clean_recipes = clean_ingredients.to_frame().join(clean_steps.to_frame())

In [570]:
clean_recipes

,ingredients,preparation_steps
title,,
Almond Butter and Banana Rice Cakes,\n- rice cakes (plain or lightly salted)\n- al...,\n1. **gather ingredients:** make sure you hav...
Apple Cinnamon Protein Muffins,"\n- 1 cup whole wheat flour (or all-purpose, o...",\n\n1. preheat your oven to 350 degrees fahren...
BBQ pulled chicken sandwiches,\n\n- 3 cups shredded cooked chicken (rotisser...,\n\n1. prepare the chicken. if you are using l...
Baba ganoush with pita bread,for pita bread (if making from scratch):\n\n-...,for baba ganoush:\n1. preheat your oven to 400...
Baked Cinnamon Sugar Pita Chips,- 4 to 6 whole wheat pita breads\n- 1/4 cup un...,1. preheat your oven to 375 degrees f (190 deg...
...,...,...
Veggie Scramble,"\n- 4 large eggs\n- salt and pepper, to taste\...","\n\n1. crack the eggs into a bowl, add salt an..."
Veggie Sushi Burrito,**sushi rice:**\n- 1 cup sushi rice (short-gra...,or use a rice cooker.\n - while the rice is...
Veggie Wrap with Avocado and Spinach,- 2 large whole wheat tortillas or wraps\n- 1 ...,"1. lay out the tortillas or wraps on a clean, ..."


In [460]:
#new_recipes_df.to_csv("new_recipes_fixed_151.csv", sep='|')

In [573]:
full_data_df = recipes_df.copy()

In [579]:
recipes_df[recipes_df['title']=='Zucchini and Sweet Potato Pancakes'].shape

(1, 9)

In [580]:
recipes_df.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation'],
      dtype='object')

In [584]:
empty_recipes = recipes_df.loc[recipes_df["ingredients"].isna(), :]

In [585]:
empty_recipes.shape

(153, 9)

In [583]:
bad_generated_recipes.shape

(20, 9)

In [593]:
to_fill_df = pd.concat([empty_recipes, bad_generated_recipes], axis=0)
to_fill_df.reset_index(inplace=True)

In [615]:
clean_rc = clean_recipes.reset_index()

In [616]:
clean_rc.drop_duplicates(subset="title", keep='first', inplace=True)

In [647]:
# update the data
for title in clean_rc['title'].tolist():
    mask = to_fill_df["title"] == title
    print(sum(mask))
    print(clean_rc.loc[clean_rc['title']==title]["ingredients"])
    to_fill_df.loc[mask, "ingredients"] = clean_rc.loc[clean_rc['title']==title, "ingredients"].values[0]
    to_fill_df.loc[mask, "preparation"] = clean_rc.loc[clean_rc['title']==title, "preparation_steps"].values[0]
    # update ingredients 


1
0    \n- rice cakes (plain or lightly salted)\n- al...
Name: ingredients, dtype: object
1
1    \n- 1 cup whole wheat flour (or all-purpose, o...
Name: ingredients, dtype: object
1
2    \n\n- 3 cups shredded cooked chicken (rotisser...
Name: ingredients, dtype: object
1
3     for pita bread (if making from scratch):\n\n-...
Name: ingredients, dtype: object
1
4    - 4 to 6 whole wheat pita breads\n- 1/4 cup un...
Name: ingredients, dtype: object
1
5    - 1 medium-sized eggplant\n- 1/2 cup all-purpo...
Name: ingredients, dtype: object
1
6    - 2 cups of dried chickpeas (soaked overnight ...
Name: ingredients, dtype: object
1
7    - 1 cup quinoa, uncooked\n- 2 cups water or ve...
Name: ingredients, dtype: object
1
8    - 2 medium-sized sweet potatoes (about 1 lb or...
Name: ingredients, dtype: object
1
9    - 2 cups dried chickpeas, soaked overnight (do...
Name: ingredients, dtype: object
1
10    **\n\nfor the balsamic marinade:\n- 1/4 cup ba...
Name: ingredients, dtype: object
1
11    -

In [646]:
clean_rc.loc[clean_rc['title']=='Vegan Banana Bread', "ingredients"].values[0]

'- 4 ripe bananas (about 1 1/3 cups mashed)\n- 1/3 cup melted coconut oil or vegetable oil\n- 1/2 cup maple syrup or agave nectar\n- 1/4 cup plant-based milk (almond, soy, oat, etc.)\n- 1 teaspoon vanilla extract\n- 2 cups all-purpose flour (for a healthier option, you can use whole wheat flour)\n- 1/2 teaspoon salt\n- 1 teaspoon baking soda\n- 1/2 teaspoon ground cinnamon (optional)\n- 1/2 cup chopped walnuts or pecans (optional)\n- 1/2 cup vegan chocolate chips or blueberries (optional)\n\n### '

In [639]:
to_fill_df.loc[0, "ingredients"] = "hola" #clean_rc.loc[clean_rc['title']=='Vegan Banana Bread']["ingredients"]

In [648]:
to_fill_df.loc[0, "ingredients"]

'- 1 block (14 to 16 oz) of firm or extra-firm tofu\n- 1 tablespoon of olive oil or your preferred cooking oil\n- 1/4 teaspoon of turmeric powder (for color)\n- 1/2 teaspoon of garlic powder (or 1-2 cloves of fresh garlic, minced)\n- 1/2 teaspoon of ground cumin (optional)\n- 1/4 teaspoon of paprika (optional)\n- salt and pepper, to taste\n- 1/4 cup of nutritional yeast (for a cheesy flavor, optional)\n- 1/4 cup of plant-based milk or water (if needed, for moisture)\n- vegetables of choice: e.g., diced onions, bell peppers, tomatoes, spinach, kale, mushrooms (quantities as desired)\n- fresh herbs like chives, parsley, or cilantro (optional, for garnish)\n- squeeze of fresh lemon juice or a dash of kala namak (black salt, for an eggy flavor, optional)\n\n'

In [654]:
filtered_recipes = full_data_df[~full_data_df["ingredients"].isna()]

In [657]:
filtered_recipes = filtered_recipes[~filtered_recipes["preparation"].isna()]

In [667]:
full_recipes = pd.concat([filtered_recipes, to_fill_df], axis=0)

In [669]:
full_recipes.tail(4)

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,index
169,Vegetarian pad see ew (thick rice noodles),52. Vegetarian pad see ew (thick rice noodles)...,dinner,halal,350,NaN,food_5809,ingredient list:\n- 14 ounces (400g) fresh wid...,\n1. **prepare the noodles:**\n - if you're ...,4945.0
170,Balsamic marinated grilled vegetables with cou...,74. Balsamic marinated grilled vegetables with...,dinner,halal,300,NaN,food_5831,**\n\nfor the balsamic marinade:\n- 1/4 cup ba...,**\n\n1. **create the marinade:**\n - in a s...,4967.0
171,Grilled honey lime chicken with quinoa and veg...,80. Grilled honey lime chicken with quinoa and...,dinner,halal,400,NaN,food_5837,for grilled honey lime chicken:**\n\n- 4 bone...,**\n\n1. **marinate the chicken:**\n - in a ...,4973.0
172,Ratatouille with crusty bread,81. Ratatouille with crusty bread\n- Calories:...,dinner,halal,350,NaN,food_5838,For the Ratatouille:\n\n1 large eggplant\n2 me...,1. Prepare the Vegetables:\n\n* Wash and peel ...,4974.0


In [670]:
unique_fixed_recipes = full_recipes.drop_duplicates(subset="title", keep='last')

In [671]:
unique_fixed_recipes.shape

(6195, 10)

In [673]:
unique_fixed_recipes.drop(columns=['index'], inplace=True)

/tmp/ipykernel_4115765/2154705007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_fixed_recipes.drop(columns=['index'], inplace=True)


In [675]:
unique_fixed_recipes.to_csv("full_recipes_6195.csv", sep="|", index=False)

In [ ]:
unique_fixed_recipes

In [680]:
# sample for text 
unique_fixed_recipes["cultural_restriction"].unique()

array(['vegan', 'vegetarian', 'halal', 'kosher', 'NotRestriction'],
      dtype=object)

In [682]:
sampling = unique_fixed_recipes.groupby('cultural_restriction', group_keys=False).apply(lambda x:
    x.sample(200))

In [684]:
sampling

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation
6654,Raspberry almond chia seed pudding,74. Raspberry almond chia seed pudding - Calor...,breakfast,NotRestriction,250,NaN,food_8172,ingredients:\n- 1 cup raspberries\n- 1 cup alm...,"\n1. in a blender, combine the raspberries and..."
6727,Cinnamon Swirl Bread,100. Cinnamon Swirl Bread:\n- Calories per por...,morning snacks,NotRestriction,300.0,NaN,food_8250,ingredients:\n- 2 cups all-purpose flour\n- 2 ...,"\n1. in a medium bowl, whisk together the flou..."
6687,Nutella Toast,28. Nutella Toast:\n- Calories per portion: 25...,morning snacks,NotRestriction,250.0,NaN,food_8208,ingredients:\n- 2 slices of bread\n- nutella\n\n,\n1. toast the bread slices until golden brown...
6661,Breakfast fried rice with veggies and eggs,81. Breakfast fried rice with veggies and eggs...,breakfast,NotRestriction,400,NaN,food_8179,sure! here's a recipe for breakfast fried rice...,\n1. heat the vegetable oil in a large skillet...
6831,Stuffed zucchini boats with ground turkey,63. Stuffed zucchini boats with ground turkey ...,dinner,NotRestriction,350.0,contains lactose,food_8413,ingredients: \n- 2 medium zucchini \n- 1 pound...,\n1. preheat your oven to 375°f (190°c). \n2. ...
...,...,...,...,...,...,...,...,...,...
2408,Garlic Bread,"39. Garlic Bread: 250 calories per portion, 1 ...",morning snacks,vegetarian,250.0,NaN,food_2661,ingredients:\n- 1 loaf of french bread\n- 1/2 ...,\n1. preheat your oven to 375°f (190°c).\n2. s...
3408,Chana masala,"38. Chana masala - 300 calories per portion, 4...",dinner,vegetarian,300.0,contains nuts (India),food_4018,ingredients:\n- 2 cups cooked chickpeas\n- 1 l...,\n1. heat the vegetable oil in a large pan ove...
2461,Chocolate Chia Seed Pudding,33. Chocolate Chia Seed Pudding: 300 calories ...,morning snacks,vegetarian,300.0,contains dairy,food_2715,ingredients:\n- 1/4 cup chia seeds\n- 1 cup mi...,"\n1. in a medium-sized bowl, combine chia seed..."
2513,Mango Chia Pudding,5. Mango Chia Pudding: (180 kcals per portion)...,morning snacks,vegetarian,180.0,NaN,food_2770,ingredients:\n- 1 ripe mango\n- 1/4 cup chia s...,\n1. peel the mango and cut the flesh into sma...


In [686]:
# prepare output for benoit 
import os 

In [687]:
base_dir = "/home/victor/Documents/Expectation_data_generation/src/outputs/16-01-2024_17-12-01"

In [695]:
user_df = pd.read_csv(os.path.join(base_dir, 'users_dataset.csv'), index_col=0)
tracking_df = pd.read_csv(os.path.join(base_dir, 'tracking.csv'), index_col=0)

In [697]:
user_df.columns

Index(['userId', 'nutrition_goal', 'clinical_gender', 'age_range',
       'life_style', 'weight', 'height', 'projected_daily_calories',
       'current_daily_calories', 'country_of_origin', 'living_country',
       'current_location', 'cultural_factor', 'probabilities', 'allergy',
       'Multi-allergy', 'current_working_status', 'marital_status',
       'ethnicity', 'BMI', 'next_BMI'],
      dtype='object')

In [698]:
user_features = user_df[['userId', 'nutrition_goal', 'clinical_gender', 'age_range',
                         'life_style', 'weight', 'height', 'projected_daily_calories',
                         'current_daily_calories', 'country_of_origin', 'living_country',
                         'allergy', 'cultural_factor', 'Multi-allergy',
                         'current_working_status', 'marital_status', 'ethnicity', 'BMI']]

In [700]:
tracking_features = tracking_df[['day_number', 
                                 'meal_type', 
                                 'userId', 
                                 'foodId',
                                 'time_of_meal_consumption', 
                                 'place_of_meal_consumption',
                                 'social_situation_of_meal_consumption']]

In [701]:
joint_dataset = user_features.merge(tracking_features, on='userId')

In [704]:
joint_dataset.rename(columns={'foodId': 'label'}, inplace=True)

In [706]:
joint_dataset = joint_dataset[['userId', 'nutrition_goal', 'clinical_gender', 'age_range',
       'life_style', 'weight', 'height', 'projected_daily_calories',
       'current_daily_calories', 'country_of_origin', 'living_country',
       'allergy', 'cultural_factor', 'Multi-allergy', 'current_working_status',
       'marital_status', 'ethnicity', 'BMI', 'day_number', 'meal_type',
       'time_of_meal_consumption', 'place_of_meal_consumption',
       'social_situation_of_meal_consumption', 'label']]

In [707]:
joint_dataset.to_csv("dataset_1000_recipes.csv", index=False)

In [699]:
tracking_df.columns

Index(['day_number', 'meal_type', 'userId', 'foodId',
       'time_of_meal_consumption', 'place_of_meal_consumption',
       'social_situation_of_meal_consumption', 'appreciation_feedback'],
      dtype='object')

In [839]:
# load second bach recipes
df_additional_raw = pd.read_csv("/home/victor/Documents/Expectation_data_generation/df_protein_raw.csv",
                            sep="|", index_col=0)

In [840]:
def preprocess_text(text:str, pattern: str):
    lower_text = text.lower()
    ans = re.findall(pattern, lower_text, re.IGNORECASE)
    return ans

In [765]:
pattern = r'\d+\.?\d*(?=g|gram|grams)'

In [766]:
partial_extractor = partial(preprocess_text, pattern=pattern)

In [841]:
extracted_results = df_additional_raw['raw_text'].apply(lambda x: partial_extractor(x))

In [842]:
# second revision 
good_rows = extracted_results.apply(lambda x: len(x) > 1)

In [843]:
sum(good_rows)

4116

In [844]:
rows_to_process = extracted_results[good_rows]

In [845]:
def extract_number(list_num: List[float]):
    last_num = list_num[-1]
    return float(last_num)

In [846]:
numeric_df = rows_to_process.apply(lambda x : extract_number(x))

In [847]:
mask = numeric_df >= 100.00

In [848]:
sum(mask)

654

In [849]:
ready_recipes = numeric_df[~mask]

In [850]:
df_additional.loc[ready_recipes.index, "protein"] = ready_recipes.values

In [851]:
df_additional

,title,raw_text,fat,fiber,protein,carbs
0,Vegan Coconut Chia Pudding,To calculate the total fat per 100g for the gi...,NaN,5.2,NaN,NaN
1,Vegan Guacamole,The total fat per 100g in the recipe is 14g.,14.0,1.4,NaN,NaN
2,Vegan Spinach Dip,The total fat per 100g from the recipe is 42.4g.,42.4,NaN,NaN,NaN
3,Vegan Tofu Stir,To calculate the total fat per 100g from the g...,NaN,0.0,10.0,NaN
4,Vegan Sweet Potato Casserole,To calculate the total fat per 100g in the rec...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
6860,Mexican stuffed bell peppers with ground beef ...,To calculate the total fat content per 100g of...,NaN,NaN,24.0,NaN
6861,Caprese quinoa salad with grilled chicken,Since the recipe does not specify the exact am...,NaN,NaN,21.4,NaN
6862,Baked coconut shrimp with mango salsa and jasm...,"To calculate the total fat per 100g, we need t...",NaN,1.2,7.3,18.4
6863,One,To determine the total fat per 100g in the rec...,NaN,NaN,9.3,9.3


In [853]:
df_additional_raw.loc[6863, "raw_text"]

'Based on the listed ingredients, it is not possible to accurately determine the total protein per 100g without specific quantities for each ingredient.'

In [864]:
unique_with_nutritional_info = pd.merge(unique_fixed_recipes, df_additional[["title","carbs", "fat", "fiber", "protein"]], on='title', 
         how='left')

In [865]:
unique_fixed_recipes.shape

(6195, 9)

In [866]:
unique_with_nutritional_info.to_csv("unique_recipes_6195_with_nutritional.csv", sep='|')

In [867]:
# load additional data 
additional_data = pd.read_csv("/home/victor/Documents/Expectation_data_generation/full_reyhan_format_old_recipes.csv",
                              sep="|")

In [868]:
additional_data.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation', 'taste',
       'cooking_style', 'meal_type_1', 'prep_time', 'price'],
      dtype='object')

In [869]:
join_df = pd.merge(unique_with_nutritional_info, additional_data[['title', 
                                                                  'taste',
                                                                  'cooking_style', 
                                                                  'meal_type_1', 
                                                                  'prep_time', 
                                                                  'price']], 
                   on='title', how='left')

In [870]:
join_df.shape

(6195, 18)

In [871]:
join_df.to_csv("df_reyhan_format_full_recipes_6195.csv")

In [879]:
load_new_batch = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/reyhan_format_new_recipes.csv", 
                             sep='|')

In [880]:
# concatenate recipes 
load_new_batch.columns

Index(['title', 'raw_text', 'Ingredients', 'Preparation', 'Carbohydrates',
       'Protein', 'Fat ', 'Fiber', 'Calories', 'Allergens', 'meal_types',
       'Country', 'taste', 'cooking_style', 'meal_type', 'preparation_time',
       'price'],
      dtype='object')

In [884]:
map_names  = {'Ingredients': 'ingredients', 
              'Preparation': 'preparation',
              'Carbohydrates': 'carbs',
              'Protein': 'protein', 
              'Fat ': 'fat', 
              'Fiber': 'fiber', 
              'Calories': 'calories', 
              'Allergens': 'allergies', 
              'meal_types': 'meal_type_1',
              'Country': 'country', 
              'preparation_time': 'prep_time'}

In [885]:
new_batch_df = load_new_batch.rename(columns=map_names)

In [886]:
final_df = pd.concat([join_df, new_batch_df], axis=0, ignore_index=True)

In [887]:
final_df.shape

(6522, 19)

In [888]:
# add 478 recipes 
indian_batch = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/indian_batch/IndianFoodDatasetCSV.csv")

In [894]:
# choose recipes 
choosen_recipes = indian_batch.sample(478)

In [895]:
choosen_recipes.columns

Index(['Srno', 'RecipeName', 'TranslatedRecipeName', 'Ingredients',
       'TranslatedIngredients', 'PrepTimeInMins', 'CookTimeInMins',
       'TotalTimeInMins', 'Servings', 'Cuisine', 'Course', 'Diet',
       'Instructions', 'TranslatedInstructions', 'URL'],
      dtype='object')

In [896]:
# choose data 
df_indian = choosen_recipes[['TranslatedRecipeName',
       'TranslatedIngredients', 
       'TotalTimeInMins', 'Cuisine', 'Course', 'Diet',
     'TranslatedInstructions']]

In [892]:
final_df.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1',
       'prep_time', 'price', 'country'],
      dtype='object')

In [898]:
# change names 
df_indian.rename(columns={
    'TranslatedRecipeName': 'title',
       'TranslatedIngredients': 'ingredients', 
       'TotalTimeInMins': 'prep_time', 'Cuisine': 'country', 'Course':'meal_type', 
       'Diet': 'meal_type_1',
     'TranslatedInstructions': 'preparation'
    
}, inplace=True)

/tmp/ipykernel_4115765/2661746753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_indian.rename(columns={


In [899]:
df_indian.head(4)

,title,ingredients,prep_time,country,meal_type,meal_type_1,preparation
5224,Penne With Creamy Pumpkin Sauce Recipe With Br...,"500 grams Broccoli,500 grams Penne pasta,2 cup...",60,Italian Recipes,Main Course,Vegetarian,To begin making the Penne with Creamy Pumpkin ...
897,Tomato Jam Recipe,"1 kg Tomato - ripe,3/4 kg Sugar,Salt - a pinch...",50,Indian,World Breakfast,Vegetarian,"To begin making the Tomato Jam recipe, wash to..."
6262,Masaura And Potato Tarkari Recipe,"1 cups Masauras,1 cup Potatoes (Aloo) - choppe...",35,Indian,Lunch,Vegetarian,To begin making the Masaura And Potato Tarkari...
6574,Potoler Dolma Recipe - Potoler Dolma Recipe,"6 Parwal - Peel and cut on the side, 15 cashew...",50,Bengali Recipes,Lunch,Vegetarian,"To make Potolare Dolma recipe, first heat 2 cu..."


In [900]:
final_7000 = pd.concat([final_df, df_indian], axis=0, ignore_index=True)

In [903]:
final_7000.reset_index(drop=True)

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type_1,prep_time,price,country
0,Fruit Salad,"1. Fruit Salad: 70 calories per portion, 4 por...",breakfast,vegan,70.0,contains fruits only,food_0,ingredients:\n- 1 apple\n- 1 banana\n- 1 orang...,\n1. wash and cut all the fruits into bite-siz...,NaN,NaN,0.0,15.0,sweet,mixed,fruit-based,NaN,2.0,NaN
1,Vegan Omelette,"5. Vegan Omelette: 300 calories per portion, 1...",breakfast,vegan,300.0,contains chickpea flour,food_4,ingredients:\n- 1 cup chickpea flour\n- 1 cup ...,"\n1. in a mixing bowl, whisk together the chic...",NaN,NaN,6.0,15.0,NaN,sauteed,veggie,20.0,2.0,NaN
2,Vegan French Toast,7. Vegan French Toast: 400 calories per portio...,breakfast,vegan,400.0,NaN,food_6,ingredients:\n- 4 slices of vegan bread\n- 1 c...,"\n1. in a shallow dish, whisk together the alm...",NaN,NaN,NaN,17.0,sweet,sauteed,vegan,25.0,1.0,NaN
3,Granola with Soy Milk,10. Granola with Soy Milk: 550 calories per po...,breakfast,vegan,550.0,contains nuts (varies by brand),food_9,ingredients:\n- 3 cups rolled oats\n- 1 cup nu...,\n1. preheat your oven to 325°f (165°c).\n2. i...,13.1,26.0,1.5,5.5,sweet,baked,grain-based,25.0,2.0,NaN
4,Vegan Waffles,"11. Vegan Waffles: 600 calories per portion, 3...",breakfast,vegan,600.0,NaN,food_10,ingredients:\n- 2 cups all-purpose flour\n- 2 ...,"\n1. in a large bowl, whisk together the flour...",NaN,19.6,NaN,NaN,sweet,baked,vegan,50.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,Crispy Oven Roasted Curried Chickpeas Recipe,NaN,Snack,NaN,NaN,NaN,NaN,"1 cup Kabuli Chana (White Chickpeas),1 teaspoo...",To begin making Crispy Oven Roasted Curried Ch...,NaN,NaN,NaN,NaN,NaN,NaN,High Protein Vegetarian,180.0,NaN,Indian
6996,Arbi Tomato Onion Sabzi (Recipe in Hindi) with...,NaN,Lunch,NaN,NaN,NaN,NaN,"12 arbi - finely chopped, 1 onion - chopped, 2...","To make tomato onion curry arabi vegetable, fi...",NaN,NaN,NaN,NaN,NaN,NaN,Vegetarian,55.0,NaN,North Indian Recipes
6997,Papad Parathas Recipe,NaN,Lunch,NaN,NaN,NaN,NaN,1-1/2 cups Whole Wheat Flour - kneaded to doug...,"To begin with Papad Parathas, firstly fry the ...",NaN,NaN,NaN,NaN,NaN,NaN,Vegetarian,40.0,NaN,Indian
6998,Gujarati Style Bhakri Recipe-A Multigrain Flat...,NaN,North Indian Breakfast,NaN,NaN,NaN,NaN,"1 cup Whole Wheat Flour,1 cup Jowar Flour (Sor...","To begin making the Bhakri's, in a wide bowl, ...",NaN,NaN,NaN,NaN,NaN,NaN,Diabetic Friendly,50.0,NaN,Maharashtrian Recipes


In [904]:
mask = final_7000["recipeId"].isna()

In [906]:
list_ids = final_7000.loc[~mask,"recipeId"].tolist()

In [907]:
len(list_ids)

6195

In [910]:
new_ids = [f"food_{i}" for i in range(7000, 7000+805)]

In [912]:
final_7000.loc[mask, "recipeId"] = new_ids

In [913]:
final_7000

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type_1,prep_time,price,country
0,Fruit Salad,"1. Fruit Salad: 70 calories per portion, 4 por...",breakfast,vegan,70.0,contains fruits only,food_0,ingredients:\n- 1 apple\n- 1 banana\n- 1 orang...,\n1. wash and cut all the fruits into bite-siz...,NaN,NaN,0.0,15.0,sweet,mixed,fruit-based,NaN,2.0,NaN
1,Vegan Omelette,"5. Vegan Omelette: 300 calories per portion, 1...",breakfast,vegan,300.0,contains chickpea flour,food_4,ingredients:\n- 1 cup chickpea flour\n- 1 cup ...,"\n1. in a mixing bowl, whisk together the chic...",NaN,NaN,6.0,15.0,NaN,sauteed,veggie,20.0,2.0,NaN
2,Vegan French Toast,7. Vegan French Toast: 400 calories per portio...,breakfast,vegan,400.0,NaN,food_6,ingredients:\n- 4 slices of vegan bread\n- 1 c...,"\n1. in a shallow dish, whisk together the alm...",NaN,NaN,NaN,17.0,sweet,sauteed,vegan,25.0,1.0,NaN
3,Granola with Soy Milk,10. Granola with Soy Milk: 550 calories per po...,breakfast,vegan,550.0,contains nuts (varies by brand),food_9,ingredients:\n- 3 cups rolled oats\n- 1 cup nu...,\n1. preheat your oven to 325°f (165°c).\n2. i...,13.1,26.0,1.5,5.5,sweet,baked,grain-based,25.0,2.0,NaN
4,Vegan Waffles,"11. Vegan Waffles: 600 calories per portion, 3...",breakfast,vegan,600.0,NaN,food_10,ingredients:\n- 2 cups all-purpose flour\n- 2 ...,"\n1. in a large bowl, whisk together the flour...",NaN,19.6,NaN,NaN,sweet,baked,vegan,50.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,Crispy Oven Roasted Curried Chickpeas Recipe,NaN,Snack,NaN,NaN,NaN,food_7800,"1 cup Kabuli Chana (White Chickpeas),1 teaspoo...",To begin making Crispy Oven Roasted Curried Ch...,NaN,NaN,NaN,NaN,NaN,NaN,High Protein Vegetarian,180.0,NaN,Indian
6996,Arbi Tomato Onion Sabzi (Recipe in Hindi) with...,NaN,Lunch,NaN,NaN,NaN,food_7801,"12 arbi - finely chopped, 1 onion - chopped, 2...","To make tomato onion curry arabi vegetable, fi...",NaN,NaN,NaN,NaN,NaN,NaN,Vegetarian,55.0,NaN,North Indian Recipes
6997,Papad Parathas Recipe,NaN,Lunch,NaN,NaN,NaN,food_7802,1-1/2 cups Whole Wheat Flour - kneaded to doug...,"To begin with Papad Parathas, firstly fry the ...",NaN,NaN,NaN,NaN,NaN,NaN,Vegetarian,40.0,NaN,Indian
6998,Gujarati Style Bhakri Recipe-A Multigrain Flat...,NaN,North Indian Breakfast,NaN,NaN,NaN,food_7803,"1 cup Whole Wheat Flour,1 cup Jowar Flour (Sor...","To begin making the Bhakri's, in a wide bowl, ...",NaN,NaN,NaN,NaN,NaN,NaN,Diabetic Friendly,50.0,NaN,Maharashtrian Recipes


In [914]:
final_7000.to_csv("recipes_df_7000.csv", sep='|')

In [915]:
final_7000.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1',
       'prep_time', 'price', 'country'],
      dtype='object')

In [916]:
# to fix with chat gpt 
calories_mask = final_7000["calories"].isna()
carbs_mask = final_7000["carbs"].isna()
fat_mask = final_7000["fat"].isna()
fiber_mask = final_7000["fiber"].isna()
protein_mask = final_7000["protein"].isna()
allergies_mask = final_7000["allergies"].isna()
taste_mask = final_7000["taste"].isna()
cooking_mask = final_7000["cooking_style"].isna()
preptime_mask = final_7000["prep_time"].isna()
price_mask = final_7000["price"].isna()
country_mask = final_7000["country"].isna()

In [918]:
sum(calories_mask)

478

In [919]:
# save them 
list_columns = [calories_mask, carbs_mask, fat_mask, fiber_mask,
protein_mask, allergies_mask, taste_mask, cooking_mask, preptime_mask, 
price_mask, country_mask]

In [ ]:
names_list = [
    "calories_mask", "carbs_mask", "fat_mask", "fiber_mask",
"protein_mask", "allergies_mask", "taste_mask", "cooking_mask", preptime_mask, 
price_mask, country_mask
]

In [ ]:
for elm in list_columns:
    

In [921]:
#
df_full = final_7000.fillna("ffill")

In [922]:
df_full.to_csv("df_recipes_full_7000.csv", sep="|")

In [749]:
# Unique fixed recipes 
print(unique_fixed_recipes.loc[0, "ingredients"])

0    ingredients:\n- 1 apple\n- 1 banana\n- 1 orang...
0    - 1 block (14 to 16 oz) of firm or extra-firm ...
Name: ingredients, dtype: object


In [757]:
unique_fixed_recipes.reset_index(drop=True, inplace=True)

In [760]:
print(unique_fixed_recipes.loc[0, "ingredients"])

ingredients:
- 1 apple
- 1 banana
- 1 orange
- 1 cup grapes
- 1 cup strawberries
- 1 cup pineapple chunks
- 1/2 cup blueberries
- 1/2 cup raspberries
- 1/4 cup honey
- 1 tablespoon lemon juice




In [746]:
print(df_additional.iloc[7, 1])

To determine the total fat per 100g, we need to calculate the fat content of each ingredient, multiply it by the quantity used, and then add them all together.

Assuming that nutritional information is not provided for each ingredient, I will provide an estimated answer based on standard values for similar ingredients.

1. Shredded romaine lettuce: 0.2g fat per 100g
2. Vegan caesar dressing: 35g fat per 100g
3. Vegan parmesan cheese: 26g fat per 100g
4. Cherry tomatoes: 0.2g fat per 100g
5. Sliced cucumber: 0.1g fat per 100g
6. Sliced black olives: 14g fat per 100g
7. Diced red onion: 0.1g fat per 100g
8. Flour tortillas: 7.4g fat per 100g

Now, let's calculate the total fat per 100g:

(1 cup shredded romaine lettuce) * (0.2g fat per 100g) +
(1/4 cup vegan caesar dressing) * (35g fat per 100g) +
(1/4 cup vegan parmesan cheese, grated) * (26g fat per 100g) +
(1/4 cup cherry tomatoes, halved) * (0.2g fat per 100g) +
(1/4 cup sliced cucumber) * (0.1g fat per 100g) +
(1/4 cup sliced black 

In [939]:
recipe_names = final_df["title"].apply(lambda x: x.lower()).tolist()

In [949]:
# load new recipes 
new_batch_recipes = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/additional_recipes/RecipeNLG_dataset.csv",
                                index_col=0)

In [950]:
check = new_batch_recipes['title'].apply(lambda x: type(x)==float)

In [951]:
new_batch_recipes.dropna(axis=0, inplace=True)

In [952]:
new_batch_recipes.drop_duplicates(subset='title', inplace=True)

In [954]:
new_batch_recipes.to_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/additional_recipes/filtered_NGD.csv")

In [955]:
new_recipes_names = new_batch_recipes['title'].apply(lambda x: x.lower()).tolist()

In [964]:
available_recipes = list(set(new_recipes_names) - set(recipe_names))

In [959]:
len(available_recipes)

1295476

In [965]:
# choosing 700 random recipes 
cr = np.random.choice(available_recipes, size=700) 

In [974]:
# choose recipes 
mask_cr = new_batch_recipes["title"].apply(lambda x: x.lower() in cr)

In [976]:
new_recipes = new_batch_recipes.loc[mask_cr, :]

In [994]:
def process_ingredients(text: str):
    # preprocess 
    new_text = text.replace("\"", "").replace("[", "").replace("]", "")
    splitted_text = new_text.split(',')
    return " \n ".join(splitted_text)

In [997]:
new_recipes["ingredients"] = new_recipes["ingredients"].apply(lambda x: process_ingredients(x))

/tmp/ipykernel_4115765/3425381400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipes["ingredients"] = new_recipes["ingredients"].apply(lambda x: process_ingredients(x))


In [998]:
new_recipes["directions"] = new_recipes["directions"].apply(lambda x: process_ingredients(x))

/tmp/ipykernel_4115765/57806532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipes["directions"] = new_recipes["directions"].apply(lambda x: process_ingredients(x))


In [1003]:
final_df.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1',
       'prep_time', 'price', 'country'],
      dtype='object')

In [1006]:
new_recipes.rename(columns= {
    'directions': 'preparation'
}, inplace=True)

/tmp/ipykernel_4115765/3206908326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_recipes.rename(columns= {


In [1008]:
new_recipes.head(4)

,title,ingredients,preparation,link,source,NER
2619,Apple Cinnamon Cake,1 (18.25 oz.) pkg. spice or cake mix \n 1 (21...,Preheat oven to 350\u00b0. \n Blend together ...,www.cookbooks.com/Recipe-Details.aspx?id=729699,Gathered,"[""spice"", ""apple pie filling"", ""eggs"", ""sugar""..."
9271,Black Bottom Banana Bars,1/2 c. butter or margarine \n 1 c. sugar \n ...,In mixing bowl \n cream butter and sugar. \n ...,www.cookbooks.com/Recipe-Details.aspx?id=338602,Gathered,"[""butter"", ""sugar"", ""egg"", ""vanilla extract"", ..."
11707,Tuna Ball,2 (7 oz.) cans white tuna \n drained \n 8 oz...,Mix all ingredients \n except pecans. \n Chi...,www.cookbooks.com/Recipe-Details.aspx?id=781489,Gathered,"[""white tuna"", ""cream cheese"", ""horseradish"", ..."
14050,Marinated Steak,1 thick flank steak or London broil (about 1 1...,Mix together the wine \n soy sauce and season...,www.cookbooks.com/Recipe-Details.aspx?id=882299,Gathered,"[""broil"", ""red wine"", ""soy sauce"", ""oregano"", ..."


In [1009]:
new_recipes = new_recipes[["title", "ingredients", "preparation"]]

In [1012]:
new_recipes.reset_index(drop=True, inplace=True)

In [1014]:
new_recipes.to_csv("new_recipes_ng.csv", sep="|")

In [1015]:
final_df.shape

(6522, 19)

In [1016]:
final_df.to_csv("final_df_6522.csv")

In [1017]:
new_recipes.columns

Index(['title', 'ingredients', 'preparation'], dtype='object')

In [1020]:
columns_to_check = [ 'calories', 'allergies', 'recipeId', 'ingredients', 
                 'preparation', 'carbs', 'fat', 'fiber', 'protein', 
                 'taste', 'cooking_style', 'meal_type_1',
                 'prep_time', 'price', 'country']

In [1022]:
def get_index_incomplete_data(df_test:pd.DataFrame, columns_to_check: List[str]):
    index_to_fix = {}
    for col in columns_to_check:
        mask = df_test[col].isna()
        index_to_fix[col] = list(df_test.index[mask])
    return index_to_fix

In [1023]:
columns_index_to_fix = get_index_incomplete_data(final_df, 
                                                 columns_to_check)

In [1025]:
columns_index_to_fix.keys()

dict_keys(['calories', 'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat', 'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1', 'prep_time', 'price', 'country'])

In [1026]:
important_cols = ['carbs', 'fat', 'fiber', 'protein']

In [1041]:
sum(final_df['ingredients'].isna())

0

In [1042]:
final_df.to_csv("final_recipes_6522_to_fix.csv", sep="|")

In [1043]:
# save dictionary of indexes
with open("dict_indexes_to_fix.json", 'w') as fp:
    json.dump(columns_index_to_fix, fp)

In [1044]:
columns_index_to_fix.keys()

dict_keys(['calories', 'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat', 'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1', 'prep_time', 'price', 'country'])

In [1318]:
# completing the file 
path = "/home/victor/Documents/Expectation_data_generation/df_cuisine_6552_raw_new.csv"
df_to_process = pd.read_csv(path, sep="|", index_col = 0)

In [1320]:
df_to_process.shape

(6495, 2)

In [1319]:
sum(final_df['cuisine'].isna())

KeyError: 'cuisine'

In [1309]:
df_to_process.shape

(854, 2)

In [1310]:
df_to_process["raw_text"][1]

'2'

In [1276]:
def extract_raw_nutritional_info(text: str, 
                                 pattern: str, 
                                 second_pattern:str=r'=[\w\W]*\d+\.?\d*(?=\s*g|\s*gram|\s*grams)'): 
    lower_text = text.lower()
    #if "for the entire recipe" in lower_text or "=" in lower_text:
    # if "=" in lower_text:
    #     numbers = re.findall(second_pattern, lower_text, re.IGNORECASE)
    # else:
    numbers = re.findall(pattern, lower_text, re.IGNORECASE)
    return numbers

In [1290]:
grams_pattern = r'\d+\.?\d*(?=\s*g|\s*gram|\s*grams)'
partial_extractor = partial(extract_raw_nutritional_info, pattern=grams_pattern)

In [1291]:
partial_numbers_df = df_to_process["raw_text"].apply(lambda x: partial_extractor(x))

In [1292]:
# 
mask_empty = partial_numbers_df.apply(lambda x: len(x) == 0)

In [1293]:
partial_data = partial_numbers_df[~mask_empty]

In [1294]:
fat_series =  partial_data.apply(lambda x: x[-1])

In [1295]:
for idx, val in fat_series.items():
    df_to_process.loc[idx, "protein"] = val

In [1321]:
dict_fat = dict(zip(df_to_process['title'], df_to_process['raw_text']))

In [1299]:
df_updated.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1',
       'prep_time', 'price', 'country'],
      dtype='object')

In [1322]:
#df_updated = final_df.copy()
for title, fat in dict_fat.items():
    mask = df_updated['title'] == title
    df_updated.loc[mask, "cuisine"] = fat

In [1323]:
sum(df_updated['cuisine'].isna())

2

In [1324]:
df_updated.to_csv("df_6552_updated_cuisine.csv", sep='|')

In [1253]:
fat_series

0          85
1        17.6
2          99
3          68
4        52.5
        ...  
1775       70
1776    86.67
1777       40
1778       70
1779       84
Name: raw_text, Length: 1726, dtype: object

In [1166]:
test = partial_numbers_df.apply(lambda x: len(x)==1)

In [1167]:
# second_extraction
se_partial_df = partial_numbers_df[test]
print(f"size: {se_partial_df.shape}")

size: (1153,)


In [1168]:
te_partial_df = partial_numbers_df[~test]

In [1169]:
len(te_partial_df)

188

In [1170]:
fourth_mask = te_partial_df.apply(lambda x: len(x) == 0)

In [1171]:
fe_partial = te_partial_df[~fourth_mask]

In [1177]:
fe_final_list = fe_partial.apply(lambda x: x[-1])

In [1224]:
len(fe_final_list)

164

In [1175]:
df_to_process.loc[1314, "raw_text"]

"To calculate the total carbohydrates for the recipe, we need to consider the carbohydrate content of each ingredient. Spices like garlic powder, paprika, salt, and black pepper have negligible carbohydrate content, so we'll focus on the cauliflower and olive oil. Olive oil is entirely fat and contains no carbohydrates. Cauliflower is the only ingredient with a significant amount of carbohydrates.\n\nA medium-sized cauliflower (5-6 inches in diameter, 580 grams) has approximately 29 grams of carbohydrates.\n\nThus, the total carbohydrates in the recipe are:\n\nCauliflower: ~29 grams\nOlive oil: 0 grams\nSpices: Negligible \n\nTotal carbohydrates: 29 grams\n\nAnswer: 29"

In [1178]:
final_check = df_to_process.loc[mask_empty, :]

In [1205]:
ing = final_df.loc[final_df['title'] == 'Vegan Creamy Broccoli Soup', "ingredients"]

In [1206]:
print(ing.values[0])

sure! here's a recipe for vegan creamy broccoli soup:

ingredients:
- 1 large head of broccoli, chopped into florets
- 1 onion, diced
- 3 cloves of garlic, minced
- 3 cups vegetable broth
- 1 cup unsweetened almond milk (or any other plant-based milk)
- 2 tablespoons nutritional yeast
- 1 tablespoon olive oil
- salt and pepper to taste




In [1207]:
final_check.loc[351, "carbs"] = 39

In [1209]:
final_check['title'].tolist()

['Vegan Spinach and Artichoke Stuffed Mushrooms',
 'Vegan Caprese Skewers',
 'Fruit Skewers',
 'Thai Veggie Spring Rolls',
 'Vegan Creamy Broccoli Soup',
 'Vegan BBQ Chickpea Pizza',
 'Tomato and feta breakfast tart',
 'Tomato and Basil Omelette',
 'Tofu Scramble Breakfast Burrito',
 'Bruschetta with Mozzarella and Sun',
 'Boiled Egg with Whole Wheat Crackers',
 'Vegan Breakfast Muffuletta',
 'Vegan Breakfast Phyllo Cups',
 'Vegan Egg McMuffin',
 'Vegan Mediterranean Breakfast Wrap',
 'Vegan Mediterranean Toast with Hummus and Olives',
 'Vegan Sausage and Veggie Skewers',
 'Vegan Breakfast Pizza Rolls',
 'Stuffed Dates',
 'Healthy Veggie Nachos',
 'Vegan Cinnamon Rolls',
 'Vegan Mushroom and Spinach Pizza',
 'Vegan Ratatouille Pastry Puffs',
 'Chia Pudding ']

In [1211]:
# recipe names for 
with open("recipes_to_postprocessing_1.json", 'w') as fp:
    json.dump(final_check['title'].tolist(), fp) 
    

In [1212]:
partial_numbers_df[test]

0       [= 97g \n\nif cheese is added (assuming 2g for...
1       [= 108g of carbohydrates\n- plant-based milk: ...
2       [= 58g\n\ntherefore, the total approximate car...
3                                    [= approximately 92]
4                                                 [= 129]
                              ...                        
1335    [= 41 grams\n\nthe total carbohydrates for the...
1336    [= 126 grams of carbs\n\nnow, this is the tota...
1338                                              [= 163]
1339    [= 12 dates * 18 grams/date = 216 grams\ntotal...
1340    [= 190g\n\ntherefore, the total carbohydrates ...
Name: raw_text, Length: 1153, dtype: object

In [1144]:
def second_extraction(text:str, pattern:str=r'\d+\.?\d*'):
    # get the final results 
    result = re.findall(pattern, text, re.IGNORECASE)
    if len(result) > 0:
        return result[-1]
    else:
        return result

In [1145]:
list_final_values = se_partial_df.apply(lambda x: second_extraction(x[0]) if len(x) > 0 else None)

In [1147]:
list_final_values

0        99
1       109
2        58
3        92
4       129
       ... 
1335     41
1336    126
1338    163
1339    235
1340    190
Name: raw_text, Length: 1152, dtype: object

In [1151]:
len(df_to_process)

1341

In [1218]:
def reformat_ingredients_and_preparation(text_obj):
    print(type(text_obj))
    if type(text_obj) == list:
        return " ".join(text_obj)
    else:
        return text_obj

In [1225]:
# final list of calories values to fullfil 
final_result = pd.concat([list_final_values, fe_final_list])

In [1233]:
df_updated_calories = update_dataframe(final_df, "carbs", final_result)

In [1235]:
sum(df_updated_calories['carbs'].isna())

3458

In [1237]:
df_updated_calories.to_csv("reformated_6552_carbs.csv", sep='|')

In [1540]:
df_new = pd.read_csv("/home/victor/Documents/Expectation_data_generation/df_allergens_700_raw_new.csv", 
                     sep="|", 
                     index_col=0)

In [1541]:
df_new.shape

(690, 2)

In [1514]:
# choosen 
mask_fiber = df_updated['fiber'].isna()

In [1515]:
titles = df_updated.loc[mask_fiber, "title"].tolist()

In [1472]:
grams_pattern = r'\d+\.?\d*(?=\s*g|\s*gram|\s*grams)'
partial_extractor = partial(extract_raw_nutritional_info, pattern=grams_pattern)

In [1510]:
mask = df_new['raw_text'].apply(lambda x: type(x) == float)

In [1516]:
df_to_process = df_new.loc[~mask,:]

In [1517]:
partial_carbs = df_to_process["raw_text"].apply(lambda x: partial_extractor(x))

In [1518]:
df_new["carbs"] = partial_carbs.apply(lambda x: x[-1] if len(x) > 0 else np.nan)

In [1519]:
dict_up = dict(zip(df_new["title"], df_new["carbs"]))

In [1520]:
for k, v in dict_up.items():
    if k in titles:
        mask = df_updated["title"] == k
        df_updated.loc[mask, "fiber"] = v

In [1521]:
sum(df_updated["fiber"].isna())

659

In [1491]:
df_updated.to_csv("df_6552_almost_ready.csv", sep="|")

In [1435]:
df_new

,title,raw_text
0,Zucchini and Parmesan Muffins,The estimated total carbohydrates for the reci...
1,Tomato Basil Quiche,The total carbohydrates for the recipe is appr...
2,Chia Seed Pancakes,The total carbohydrates for the recipe is appr...
3,Veggie Breakfast Pizza,The estimated total carbohydrates for the enti...
4,Veggie Breakfast Wrap,The estimated total carbohydrates for the give...
...,...,...
4446,Lebanese markouk bread with za'atar and olive ...,The total carbohydrates for the given recipe w...
4447,South African melktert (milk tart) with cinnam...,The total carbohydrates for the South African ...
4448,Vietnamese bánh cuốn (steamed rice rolls) with...,The total carbohydrates in the given recipe wo...
4449,Healthy Fruit Salad,The total carbohydrates for the given recipe i...


## process the 700 

In [1328]:
df_ng = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/new_recipes_ng.csv", 
                    sep='|', 
                    index_col=0)

In [1329]:
df_ng.columns

Index(['title', 'ingredients', 'preparation'], dtype='object')

In [1330]:
df_ng.shape

(705, 3)

In [1551]:
df_to_process = pd.read_csv("/home/victor/Documents/Expectation_data_generation/df_meal_type_raw_new.csv", 
                            sep='|',
                            index_col=0)

In [1543]:
df_to_process.shape

(690, 2)

In [1528]:
def get_prep_time(text):
    try:
        if type(text) == str:
            lower_text = text.lower()
            find_all = re.findall(r'\d+\.?\d*', lower_text, re.IGNORECASE)
            if len(find_all) == 0:
                np.nan
            else:
                number = float(find_all[-1])
                if number < 3.0:
                    return number * 60.0
                else:
                    return number
        elif text.isdigit():
            if float(text) < 3.0:
                return float(text)*60.0
            else:
                return float(text)
    except Exception as e:
        print(f"Error: {e}")
    

In [1529]:
prep_time =  df_to_process["raw_text"].apply(lambda x: get_prep_time(x)) 

In [1531]:
df_ng["prep_time"] = prep_time

In [1339]:
def get_calories(text:str):
    lower_text = text.lower()
    lower_text = lower_text.replace(",", "")
    result = re.findall(r'\d+\.?\d*', lower_text, re.IGNORECASE)
    if len(result) > 0:
        return result[-1]

In [1364]:
grams_pattern = r'\d+\.?\d*(?=\s*g|\s*gram|\s*grams)'
partial_extractor = partial(extract_raw_nutritional_info, pattern=grams_pattern)

In [1375]:
df_700_partial = df_to_process['raw_text'].apply(lambda x: partial_extractor(x))

In [1376]:
mask_list = df_700_partial.apply(lambda x: len(x) > 0)

In [1377]:
df_700_carbs = df_700_partial[mask_list].apply(lambda x: x[-1])

In [1552]:
df_to_process.shape

(704, 2)

In [1405]:
df_700_carbs =  df_to_process["raw_text"]

In [1398]:
for i, j in df_700_carbs.items():
    df_ng.loc[i, "cuisine"] = j

In [1346]:
#df_ng["calories"] = df_to_process['raw_text'].apply(lambda x: get_calories(x))

In [1553]:
dict_answer = dict(zip(df_to_process["title"], df_to_process["raw_text"]))

In [1554]:
for k, v in dict_answer.items():
    mask = df_ng['title'] == k 
    df_ng.loc[mask, "meal_type"] = v

In [1555]:
df_ng.to_csv("df_700_new_recipes_rehyan_style.csv", sep='|')

In [1556]:
df_ng.head(4)

,title,ingredients,preparation,calories,carbs,fat,fiber,protein,taste,price,cooking_style,cuisine,prep_time,meal_type,allergies
0,Apple Cinnamon Cake,1 (18.25 oz.) pkg. spice or cake mix \n 1 (21...,Preheat oven to 350\u00b0. \n Blend together ...,3357.,706,55,69.2,38,Sweet,2,Slow-cooked,American,45.0,Dessert,NaN
1,Black Bottom Banana Bars,1/2 c. butter or margarine \n 1 c. sugar \n ...,In mixing bowl \n cream butter and sugar. \n ...,2681.,435.5,51,13.8,18,sweet,2,Baked,American,60.0,Vegetarian,Milk
2,Tuna Ball,2 (7 oz.) cans white tuna \n drained \n 8 oz...,Mix all ingredients \n except pecans. \n Chi...,913.,45,36,30.6,119,Salty,2,Sauteed,American,10.0,Meat-based,Tree nuts
3,Marinated Steak,1 thick flank steak or London broil (about 1 1...,Mix together the wine \n soy sauce and season...,1215.,5,56,27,176,Sweet,2,Assembled,Italian,60.0,Meat-based,Soybeans


In [1538]:
df_updated.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1',
       'prep_time', 'price', 'country', 'cuisine'],
      dtype='object')

In [1550]:
df_ng.columns

Index(['title', 'ingredients', 'preparation', 'calories', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'price', 'cooking_style', 'cuisine',
       'prep_time', 'meal_type', 'allergies'],
      dtype='object')

In [ ]:
# create final dataset 

In [1351]:
# get carbs 
df_to_process['raw_text'][0]

"To calculate the total carbohydrates in this recipe, we will need to find the carbohydrate content of each ingredient and then add them together. Since the exact nutritional content for specific brands of ingredients, like spice or cake mix and apple pie filling, are not provided, I'll use general averages. \n\nKeep in mind that actual carbohydrate content may vary based on the brands and products used.\n\n1. Spice or Cake Mix (18.25 oz.): Depending on the brand, one package of dry cake mix typically contains around 36-52 grams of carbs per serving, with about 10 servings per package. Let's use an average: 44 grams x 10 servings = 440 grams of carbohydrates.\n\n2. Apple Pie Filling (21 oz.): The carbohydrate content for apple pie filling can widely vary, but it typically contains around 20-30 grams of carbs per 1/3 cup serving. There are roughly 3 cups in 21 oz., or about 9 servings of 1/3 cup each. Using an average of 25 grams: 25 grams x 9 servings = 225 grams of carbohydrates.\n\n3

In [1387]:
import glob

In [1407]:
import requests

In [1408]:
api_key = 'v7Xyli94PjQtMlzhIqEsfA==SUxH301I99fnLlXC'

In [1501]:
mask = df_updated['fiber'].isna()

In [1502]:
# Missed values 
df_to_fix = df_updated.loc[mask, :]

In [1494]:
df_to_fix.shape

(1292, 20)

In [1495]:
df_to_fix.to_csv("df_to_fix_fiber.csv", sep='|')

In [1414]:
df_to_fix["ingredients"][0]

'ingredients:\n- 1 apple\n- 1 banana\n- 1 orange\n- 1 cup grapes\n- 1 cup strawberries\n- 1 cup pineapple chunks\n- 1/2 cup blueberries\n- 1/2 cup raspberries\n- 1/4 cup honey\n- 1 tablespoon lemon juice\n\n'

In [1415]:
query = """
'ingredients:\n- 1 apple\n- 1 banana\n- 1 orange\n- 1 cup grapes\n- 1 cup strawberries\n
- 1 cup pineapple chunks\n- 1/2 cup blueberries\n- 1/2 cup raspberries\n- 1/4 cup honey\n
- 1 tablespoon lemon juice\n\n'
"""
api_url = 'https://api.api-ninjas.com/v1/nutrition?query={}'.format(query)
response = requests.get(api_url, headers={'X-Api-Key': f'{api_key}'})
if response.status_code == requests.codes.ok:
    print(response.text)
else:
    print("Error:", response.status_code, response.text)

[{"name": "pineapple", "calories": 126.9, "serving_size_g": 250.0, "fat_total_g": 0.3, "fat_saturated_g": 0.0, "protein_g": 1.4, "sodium_mg": 2, "potassium_mg": 20, "cholesterol_mg": 0, "carbohydrates_total_g": 32.6, "fiber_g": 3.6, "sugar_g": 24.7}, {"name": "lemon", "calories": 4.1, "serving_size_g": 14.175, "fat_total_g": 0.0, "fat_saturated_g": 0.0, "protein_g": 0.2, "sodium_mg": 0, "potassium_mg": 2, "cholesterol_mg": 0, "carbohydrates_total_g": 1.3, "fiber_g": 0.4, "sugar_g": 0.4}]


In [1523]:
# Join datasets 
df_updated

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'cooking_style', 'meal_type_1',
       'prep_time', 'price', 'country', 'cuisine'],
      dtype='object')

In [1557]:
df_ng.columns

Index(['title', 'ingredients', 'preparation', 'calories', 'carbs', 'fat',
       'fiber', 'protein', 'taste', 'price', 'cooking_style', 'cuisine',
       'prep_time', 'meal_type', 'allergies'],
      dtype='object')

In [1559]:
df_ng.rename(columns={'meal_type': 'meal_type_1'}, inplace=True)

In [1560]:
df_total_ds = pd.concat([df_updated, df_ng], ignore_index=True)

In [1561]:
df_total_ds

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type_1,prep_time,price,country,cuisine
0,Fruit Salad,"1. Fruit Salad: 70 calories per portion, 4 por...",breakfast,vegan,70.0,contains fruits only,food_0,ingredients:\n- 1 apple\n- 1 banana\n- 1 orang...,\n1. wash and cut all the fruits into bite-siz...,223,2,0.0,15.0,sweet,mixed,fruit-based,NaN,2.0,NaN,Fruit Salad - Mixed
1,Vegan Omelette,"5. Vegan Omelette: 300 calories per portion, 1...",breakfast,vegan,300.0,contains chickpea flour,food_4,ingredients:\n- 1 cup chickpea flour\n- 1 cup ...,"\n1. in a mixing bowl, whisk together the chic...",100,20.5,6.0,15.0,Savory,sauteed,veggie,20.0,2.0,NaN,Vegan Omelette - Vegan
2,Vegan French Toast,7. Vegan French Toast: 400 calories per portio...,breakfast,vegan,400.0,NaN,food_6,ingredients:\n- 4 slices of vegan bread\n- 1 c...,"\n1. in a shallow dish, whisk together the alm...",115,38.5,11.8,17.0,sweet,sauteed,vegan,25.0,1.0,NaN,Vegan French Toast: French
3,Granola with Soy Milk,10. Granola with Soy Milk: 550 calories per po...,breakfast,vegan,550.0,contains nuts (varies by brand),food_9,ingredients:\n- 3 cups rolled oats\n- 1 cup nu...,\n1. preheat your oven to 325°f (165°c).\n2. i...,13.1,26.0,1.5,5.5,sweet,baked,grain-based,25.0,2.0,NaN,Breakfast
4,Vegan Waffles,"11. Vegan Waffles: 600 calories per portion, 3...",breakfast,vegan,600.0,NaN,food_10,ingredients:\n- 2 cups all-purpose flour\n- 2 ...,"\n1. in a large bowl, whisk together the flour...",382,19.6,3.6,2.6,sweet,baked,vegan,50.0,2.0,NaN,Vegan Waffles: Vegan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,Smoky Salmon with Miso-Dressed Vegetables,NaN,NaN,NaN,2106,NaN,NaN,1/4 cup white miso \n 1/4 cup chicken stock \...,In a saucepan \n cook the miso \n chicken st...,NaN,432,NaN,NaN,Salty,Baked,Meat-based,30.0,2,NaN,French
7223,Lobster of the Links Napa Cabbage Wraps,NaN,NaN,NaN,5093.,Shellfish,NaN,1 pound butter \n 4 (4-ounce) lobster tails \...,To begin the lobster filling: Melt the butter ...,NaN,292,NaN,NaN,sweet,Baked,Meat-based,60.0,2,NaN,Salad
7224,Fresh Gingerbread with Port-Poached Pears,NaN,NaN,NaN,7653.6,NaN,NaN,1/4 cup lemon juice fresh \n 8 large pears fi...,FOR PEARS: Add lemon juice to large bowl of co...,NaN,10,NaN,NaN,Salty,Baked,dessert,45.0,2,NaN,Mediterranean
7225,Jamocha Bread,NaN,NaN,NaN,2128,NaN,NaN,1 cup strong coffee (quality blend and brewed ...,Warm the coffee to about 100 degrees (30 secon...,NaN,124,NaN,NaN,Sweet,Baked,Beverage,30.0,2,NaN,Soup.


In [1562]:
df_total_ds.to_csv('df_total_recipes_almost_ready_data.csv', sep='|')

In [1563]:
mask_list = df_total_ds['recipeId'].isna()

In [1565]:
list_values = [f"food_{i}" for i in range(7000, 7000+sum(mask_list))]

In [1566]:
len(list_values)

1032

In [1567]:
df_total_ds.loc[mask_list, 'recipeId'] = list_values

In [1569]:
# view for 
df_almost_total = df_total_ds[['title', 
             'raw_text', 
             'cultural_restriction', 
             'calories',
             'allergies', 
             'recipeId', 
             'ingredients', 
             'preparation', 
             'carbs', 
             'fat',
             'fiber', 
             'protein', 
             'taste', 
             'cooking_style', 
             'meal_type_1',
             'prep_time', 
             'cuisine',
             'price'
]]

In [1571]:
df_almost_total.reset_index(drop=True, inplace=True)

In [1572]:
df_almost_total.rename(columns={'meal_type_1': 'meal_type'}, inplace=True)

/tmp/ipykernel_4115765/2235655350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_almost_total.rename(columns={'meal_type_1': 'meal_type'}, inplace=True)


In [1573]:
df_almost_total

,title,raw_text,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type,prep_time,cuisine,price
0,Fruit Salad,"1. Fruit Salad: 70 calories per portion, 4 por...",vegan,70.0,contains fruits only,food_0,ingredients:\n- 1 apple\n- 1 banana\n- 1 orang...,\n1. wash and cut all the fruits into bite-siz...,223,2,0.0,15.0,sweet,mixed,fruit-based,NaN,Fruit Salad - Mixed,2.0
1,Vegan Omelette,"5. Vegan Omelette: 300 calories per portion, 1...",vegan,300.0,contains chickpea flour,food_4,ingredients:\n- 1 cup chickpea flour\n- 1 cup ...,"\n1. in a mixing bowl, whisk together the chic...",100,20.5,6.0,15.0,Savory,sauteed,veggie,20.0,Vegan Omelette - Vegan,2.0
2,Vegan French Toast,7. Vegan French Toast: 400 calories per portio...,vegan,400.0,NaN,food_6,ingredients:\n- 4 slices of vegan bread\n- 1 c...,"\n1. in a shallow dish, whisk together the alm...",115,38.5,11.8,17.0,sweet,sauteed,vegan,25.0,Vegan French Toast: French,1.0
3,Granola with Soy Milk,10. Granola with Soy Milk: 550 calories per po...,vegan,550.0,contains nuts (varies by brand),food_9,ingredients:\n- 3 cups rolled oats\n- 1 cup nu...,\n1. preheat your oven to 325°f (165°c).\n2. i...,13.1,26.0,1.5,5.5,sweet,baked,grain-based,25.0,Breakfast,2.0
4,Vegan Waffles,"11. Vegan Waffles: 600 calories per portion, 3...",vegan,600.0,NaN,food_10,ingredients:\n- 2 cups all-purpose flour\n- 2 ...,"\n1. in a large bowl, whisk together the flour...",382,19.6,3.6,2.6,sweet,baked,vegan,50.0,Vegan Waffles: Vegan,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,Smoky Salmon with Miso-Dressed Vegetables,NaN,NaN,2106,NaN,food_8027,1/4 cup white miso \n 1/4 cup chicken stock \...,In a saucepan \n cook the miso \n chicken st...,NaN,432,NaN,NaN,Salty,Baked,Meat-based,30.0,French,2
7223,Lobster of the Links Napa Cabbage Wraps,NaN,NaN,5093.,Shellfish,food_8028,1 pound butter \n 4 (4-ounce) lobster tails \...,To begin the lobster filling: Melt the butter ...,NaN,292,NaN,NaN,sweet,Baked,Meat-based,60.0,Salad,2
7224,Fresh Gingerbread with Port-Poached Pears,NaN,NaN,7653.6,NaN,food_8029,1/4 cup lemon juice fresh \n 8 large pears fi...,FOR PEARS: Add lemon juice to large bowl of co...,NaN,10,NaN,NaN,Salty,Baked,dessert,45.0,Mediterranean,2
7225,Jamocha Bread,NaN,NaN,2128,NaN,food_8030,1 cup strong coffee (quality blend and brewed ...,Warm the coffee to about 100 degrees (30 secon...,NaN,124,NaN,NaN,Sweet,Baked,Beverage,30.0,Soup.,2


In [1574]:
df_almost_total.to_csv("df_recipes_7000_updated.csv", sep='|')

In [5]:
df_almost_total = pd.read_csv("df_recipes_7000_updated.csv", sep='|', index_col=0)

In [6]:
df_almost_total

,title,raw_text,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type,prep_time,cuisine,price
0,Fruit Salad,"1. Fruit Salad: 70 calories per portion, 4 por...",vegan,70.0,contains fruits only,food_0,ingredients:\n- 1 apple\n- 1 banana\n- 1 orang...,\n1. wash and cut all the fruits into bite-siz...,223.0,2.0,0.0,15.0,sweet,mixed,fruit-based,NaN,Fruit Salad - Mixed,2.0
1,Vegan Omelette,"5. Vegan Omelette: 300 calories per portion, 1...",vegan,300.0,contains chickpea flour,food_4,ingredients:\n- 1 cup chickpea flour\n- 1 cup ...,"\n1. in a mixing bowl, whisk together the chic...",100.0,20.5,6.0,15.0,Savory,sauteed,veggie,20.0,Vegan Omelette - Vegan,2.0
2,Vegan French Toast,7. Vegan French Toast: 400 calories per portio...,vegan,400.0,NaN,food_6,ingredients:\n- 4 slices of vegan bread\n- 1 c...,"\n1. in a shallow dish, whisk together the alm...",115.0,38.5,11.8,17.0,sweet,sauteed,vegan,25.0,Vegan French Toast: French,1.0
3,Granola with Soy Milk,10. Granola with Soy Milk: 550 calories per po...,vegan,550.0,contains nuts (varies by brand),food_9,ingredients:\n- 3 cups rolled oats\n- 1 cup nu...,\n1. preheat your oven to 325°f (165°c).\n2. i...,13.1,26.0,1.5,5.5,sweet,baked,grain-based,25.0,Breakfast,2.0
4,Vegan Waffles,"11. Vegan Waffles: 600 calories per portion, 3...",vegan,600.0,NaN,food_10,ingredients:\n- 2 cups all-purpose flour\n- 2 ...,"\n1. in a large bowl, whisk together the flour...",382.0,19.6,3.6,2.6,sweet,baked,vegan,50.0,Vegan Waffles: Vegan,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,Smoky Salmon with Miso-Dressed Vegetables,NaN,NaN,2106.0,NaN,food_8027,1/4 cup white miso \n 1/4 cup chicken stock \...,In a saucepan \n cook the miso \n chicken st...,NaN,432.0,NaN,NaN,Salty,Baked,Meat-based,30.0,French,2
7223,Lobster of the Links Napa Cabbage Wraps,NaN,NaN,5093.0,Shellfish,food_8028,1 pound butter \n 4 (4-ounce) lobster tails \...,To begin the lobster filling: Melt the butter ...,NaN,292.0,NaN,NaN,sweet,Baked,Meat-based,60.0,Salad,2
7224,Fresh Gingerbread with Port-Poached Pears,NaN,NaN,7653.6,NaN,food_8029,1/4 cup lemon juice fresh \n 8 large pears fi...,FOR PEARS: Add lemon juice to large bowl of co...,NaN,10.0,NaN,NaN,Salty,Baked,dessert,45.0,Mediterranean,2
7225,Jamocha Bread,NaN,NaN,2128.0,NaN,food_8030,1 cup strong coffee (quality blend and brewed ...,Warm the coffee to about 100 degrees (30 secon...,NaN,124.0,NaN,NaN,Sweet,Baked,Beverage,30.0,Soup.,2


In [10]:
cols = df_almost_total.columns

In [12]:
missing_values = {}
for col in cols:
    missing_values[col] = df_almost_total[col].isna()
    print(f"Col: {col} total nans: {sum(missing_values[col])}")

Col: title total nans: 0
Col: raw_text total nans: 705
Col: cultural_restriction total nans: 1032
Col: calories total nans: 647
Col: allergies total nans: 5068
Col: recipeId total nans: 0
Col: ingredients total nans: 0
Col: preparation total nans: 15
Col: carbs total nans: 80
Col: fat total nans: 193
Col: fiber total nans: 767
Col: protein total nans: 210
Col: taste total nans: 2
Col: cooking_style total nans: 780
Col: meal_type total nans: 759
Col: prep_time total nans: 1723
Col: cuisine total nans: 5
Col: price total nans: 1


In [15]:
# fix empty cols 
for k, v in missing_values.items():
    total_elements_to_fix = sum(v)
    if total_elements_to_fix > 0:
        df_selected = df_almost_total.loc[v, :]
        print(f"For column: {k} renew_df shape: {df_selected.shape}")
        df_selected.to_csv(f"df_fix_final_col_{k}.csv", sep='|')


For column: raw_text renew_df shape: (705, 18)
For column: cultural_restriction renew_df shape: (1032, 18)
For column: calories renew_df shape: (647, 18)
For column: allergies renew_df shape: (5068, 18)
For column: preparation renew_df shape: (15, 18)
For column: carbs renew_df shape: (80, 18)
For column: fat renew_df shape: (193, 18)
For column: fiber renew_df shape: (767, 18)
For column: protein renew_df shape: (210, 18)
For column: taste renew_df shape: (2, 18)
For column: cooking_style renew_df shape: (780, 18)
For column: meal_type renew_df shape: (759, 18)
For column: prep_time renew_df shape: (1723, 18)
For column: cuisine renew_df shape: (5, 18)
For column: price renew_df shape: (1, 18)


In [30]:
recipes_to_fix = df_almost_total.loc[missing_values['price'], :]

In [31]:
recipes_to_fix

,title,raw_text,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type,prep_time,cuisine,price
7226,Roast Beef With Potato Salad,NaN,NaN,2180.0,NaN,food_8031,2 lb. red potatoes \n 1 small bunch of chives...,Place potatoes and water to cover in saucepan;...,NaN,NaN,NaN,NaN,sweet,NaN,Meat-based,30.0,NaN,NaN


In [26]:
recipes_to_fix['ingredients'].str.contains('')

'i apologize, but i am an ai language model and cannot provide specific recipes that i have not been trained on. however, i can offer a general recipe for vegan raspberry almond scones. here it is:\n\ningredients:\n- 1 ½ cups all-purpose flour (gluten-free flour can be substituted if desired)\n- 2 tablespoons almond meal\n- 2 teaspoons baking powder\n- 1/4 teaspoon salt\n- 1/4 cup cold vegan butter, cut into small pieces\n- 3 tablespoons maple syrup or agave nectar\n- 1/2 cup unsweetened almond milk\n- 1 teaspoon vanilla extract\n- 1/2 cup fresh or frozen raspberries\n- 2 tablespoons sliced almonds\n\n'

In [29]:
recipes_to_fix.head(2)

,title,raw_text,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type,prep_time,cuisine,price
167,Vegan Raspberry Almond Scones,50. Vegan Raspberry Almond Scones - 250 calori...,vegan,250.0,NaN,food_220,"i apologize, but i am an ai language model and...",\n1. preheat your oven to 425°f (220°c). line ...,NaN,65.0,2.4,18.0,sweet,baked,vegan,NaN,Vegan,2.0
378,Vegan Pumpkin Spice Pancakes,76. Vegan Pumpkin Spice Pancakes\nCalories per...,vegan,NaN,NaN,food_504,ingredients:\n- 1 cup pumpkin puree\n- 1 3/4 c...,"\n1. in a large bowl, whisk together the pumpk...",NaN,20.0,15.0,25.0,sweet,sauteed,vegan,35.0,Vegan,2.0


In [38]:
print(recipes_to_fix['ingredients'][7226])

2 lb. red potatoes 
  1 small bunch of chives 
  1/2 c. sour cream 
  2 tbsp. white horseradish 
  1 tsp. salt 
  .13 tsp. Pepper 
  1 minced shallot 
  1 1/2 lb. deli roast beef


In [39]:
df_almost_total.loc[7226, 'price'] = 2

In [41]:
# checking for empty values 
df_almost_total.loc[7226, :]

title                                        Roast Beef With Potato Salad
raw_text                                                              NaN
cultural_restriction                                                  NaN
calories                                                           2180.0
allergies                                                             NaN
recipeId                                                        food_8031
ingredients             2 lb. red potatoes \n  1 small bunch of chives...
preparation             Place potatoes and water to cover in saucepan;...
carbs                                                                 NaN
fat                                                                   NaN
fiber                                                                 NaN
protein                                                               NaN
taste                                                               sweet
cooking_style                         

In [42]:
# fixed some columns 
mask_raw = df_almost_total['raw_text'].isna()

In [67]:
def fill_raw(row:pd.Series):
    text = ""
    try:
        text = row['title']+row['ingredients']+row['preparation']
    except Exception as e:
        print(f"Error: {e}")
    return text

In [68]:
fill_raw(df_almost_total.loc[0, :])  

'Fruit Saladingredients:\n- 1 apple\n- 1 banana\n- 1 orange\n- 1 cup grapes\n- 1 cup strawberries\n- 1 cup pineapple chunks\n- 1/2 cup blueberries\n- 1/2 cup raspberries\n- 1/4 cup honey\n- 1 tablespoon lemon juice\n\n\n1. wash and cut all the fruits into bite-sized pieces.\n2. place the apple, banana, orange, grapes, strawberries, pineapple chunks, blueberries, and raspberries in a large mixing bowl.\n3. in a separate small bowl, mix honey and lemon juice together until well combined.\n4. pour the honey and lemon juice mixture over the fruits.\n5. gently toss the fruits with the honey and lemon juice mixture until all fruits are evenly coated.\n6. serve the fruit salad immediately or refrigerate for later use.\n7. enjoy your delicious fruit salad!'

In [74]:
df_almost_total['raw_text'] = df_almost_total['raw_text'].astype("string")


In [78]:
sum(df_almost_total['calories'].isna())

647

In [118]:
path = "/home/victor/Documents/Expectation_data_generation/df_allergies_fixed_raw_new.csv"
df_data = pd.read_csv(path, sep='|', index_col=0)

In [133]:
dict_to_fill = dict(zip(df_data['title'].tolist(), df_data['raw_text'].tolist()))

In [135]:
for k, v in dict_to_fill.items():
    mask = df_almost_total['title'] == k
    df_almost_total.loc[mask, 'allergies'] = v    

In [122]:
dict_allergies = {
    'milk': ['milk', 'lactose', 'dairy', 'cream', 'butter', 'yogurt', 'cheese'], 
    'eggs': ['egg', 'eggs'], 
    'fish': ['bass', 'flounder', 'cod', 'fish'], 
    'shellfish': ['crustacean', 'shellfish', 'crab', 'lobster', 'shrimp'],
    'nuts': ['tree nuts', 'nuts', 'almonds', 'walnuts', 'pecans'],
    'peanuts': ['peanuts'],
    'wheat': ['wheat', 'gluten', 'durum', 'gluten', 'semolina', 'spelt'], 
    'soybeans': ['soybeans', 'soy'], 
    'sesame': ['sesame']
}

In [139]:
def extract_allergies(text: str, dict_allergens: Dict[str, List[str]]):
    lower_text = text.lower()
    allergens_words = []
    for k, v in dict_allergens.items():
        words = '|'.join(v)
        regex = r'\b(?:{})\b'
        test = re.search(regex.format(words), lower_text)
        if test is not None:
            allergens_words.append(k)
    if len(allergens_words) > 0:
        return ";".join(allergens_words)
    else:
        return lower_text

In [140]:
partial_allergens = partial(extract_allergies, 
                            dict_allergens = dict_allergies)

In [141]:
partial_allergens('contains Nuts (varies by brand)')

'nuts'

In [157]:
mask = df_almost_total['allergies'].isna()
sum(mask)

0

In [146]:
df_almost_total.loc[~mask, 'allergies'] = df_almost_total.loc[~mask, 'allergies'].apply(lambda x: partial_allergens(x))

In [162]:
sum(df_almost_total['title'].str.contains(','))

706

In [171]:
def long_word_detector(text:str, sup_limit:int=15):
    n_words = len(text.split(" "))
    if n_words > sup_limit:
        return True
    else:
        return False

In [ ]:
def fix_titles(text: str):
    # fix titles at the end once all the data have been updated. 
    re.sub(r'\(\d+[\w|\W]+\)', '', )

In [192]:
def detect_incoherences(text: str):
    lower_text = text.lower()
    if 'apologize' in lower_text or 'sorry' in lower_text:
        return True
    else:
        return False

In [193]:
mask = df_almost_total['ingredients'].apply(lambda x: detect_incoherences(x))

In [194]:
sum(mask)

10

In [200]:
mask = df_almost_total['preparation'].isna()
sum(mask)

15

In [202]:
df_to_fix = df_almost_total.loc[mask, :]

In [203]:
df_to_fix.to_csv("missing_instructions.csv", sep='|')

In [206]:
# load new instructions 
df_new_data = pd.read_excel("/home/victor/Documents/Expectation_data_generation/src/meals_collection/missing_instructions.xlsx",
                            index_col=0)

In [207]:
df_new_data

,title,raw_text,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,carbs,fat,fiber,protein,taste,cooking_style,meal_type,prep_time,cuisine,price
6198,**Beef and Broccoli - Lunch/Dinner**,8. **Beef and Broccoli - Lunch/Dinner**\n - ...,NaN,160.0,soybeans,food_7003,"Ingredients:\n\n1 pound (450g) flank steak, th...","Instructions:\n\n1. In a small bowl, whisk tog...",12.0,6.0,2.0,17.0,salty,sauteed,lunch;dinner,NaN,Chinese,2.0
6201,**Char Siu (BBQ Pork) - Lunch/Dinner**,10. **Char Siu (BBQ Pork) - Lunch/Dinner**\n ...,NaN,230.0,wheat;soybeans,food_7006,Ingredients:\n\n1 pound (450g) pork shoulder o...,"Instructions:\n\n1. In a bowl, whisk together ...",25.0,8.0,0.0,27.0,salty,grilled.,lunch;dinner,NaN,Chinese,2.0
6204,**Congee (Rice Porridge) - Breakfast**,1. **Congee (Rice Porridge) - Breakfast**\n ...,NaN,100.0,none (unless meat or additional ingredients co...,food_7009,Ingredients:\n\n1 cup rice (jasmine or long-gr...,Instructions:\n\n1. Rinse the rice thoroughly ...,20.0,1.0,0.2,3.0,salty,slow-cooked,breakfast,NaN,Chinese,2.0
6206,**Dan Dan Noodles - Lunch/Dinner**,11. **Dan Dan Noodles - Lunch/Dinner**\n - ...,NaN,180.0,wheat;soybeans,food_7011,Ingredients:\n\nFor the Sauce:\n\n2 tablespoon...,Instructions:\n\n1. Cook the Chinese egg noodl...,24.0,6.0,2024-05-01 00:00:00,9.0,salty,sauteed,lunch;dinner,NaN,Chinese,2.0
6210,**Egg Fried Rice - Lunch/Dinner**,2. **Egg Fried Rice - Lunch/Dinner**\n - Ing...,NaN,200.0,eggs;soybeans,food_7015,Ingredients:\n\n3 cups cooked and cooled rice ...,Instructions:\n\n1. Heat the vegetable oil in ...,30.0,5.0,1.0,5.0,salty,fried,lunch;dinner,NaN,Chinese,2.0
6218,**Hot and Sour Soup - Lunch/Dinner**,5. **Hot and Sour Soup - Lunch/Dinner**\n - ...,NaN,50.0,eggs;soybeans,food_7023,Ingredients:\n\n6 cups chicken or vegetable br...,"Instructions:\n\n1. In a large pot, bring the ...",5.0,2.0,1.0,3.0,sour,simmered,lunch;dinner,NaN,Chinese,2.0
6223,**Kung Pao Chicken - Lunch/Dinner**,7. **Kung Pao Chicken - Lunch/Dinner**\n - I...,NaN,240.0,peanuts;soybeans,food_7028,Ingredients:\n\nFor the Chicken Marinade:\n\n1...,"Instructions:\n\n1. In a bowl, combine the chi...",18.0,12.0,2.0,19.0,salty,sauteed,lunch;dinner,NaN,Chinese,2.0
6227,**Mapo Tofu - Lunch/Dinner**,4. **Mapo Tofu - Lunch/Dinner**\n - Ingredie...,NaN,150.0,soybeans,food_7032,\nCertainly! Here are the preparation instruct...,Instructions:\n\n1. Heat the vegetable oil in ...,9.0,7.0,1.0,11.0,salty,sauteed,lunch;dinner,NaN,Chinese,2.0
6229,**Peking Duck - Dinner**,9. **Peking Duck - Dinner**\n - Ingredients:...,NaN,260.0,wheat;soybeans,food_7034,Ingredients:\n\nWhole duck (4-5 pounds)\n1/4 c...,Instructions:\n\n1. Clean and rinse the duck t...,11.0,19.0,0.5,15.0,umami,roasted,dinner,NaN,Chinese,2.0
6238,**Scallion Pancakes - Breakfast/Lunch**,12. **Scallion Pancakes - Breakfast/Lunch**\n ...,NaN,360.0,wheat,food_7043,Ingredients:\n\n2 cups all-purpose flour\n1 cu...,"Instructions:\n\n1. In a mixing bowl, add the ...",45.0,20.0,2024-05-02 00:00:00,6.0,salty,fried,breakfast;lunch,NaN,Chinese,1.0


In [225]:
dict_replace = dict(zip(df_new_data.index, df_new_data['ingredients'].tolist()))

In [258]:
def process_price(text: str):
    if type(text) == str:
        lower_text = text.lower()
        if lower_text.isnumeric() or lower_text.isdecimal():
            return float(lower_text)
        else:
            result = re.findall(r'\d\.?\d*', lower_text)
            if len(result) == 1:
                return float(result[0])
            elif len(result) > 1:
                return float(result[-1])
            else:
                return np.nan
    elif type(text) == int or type(text) == float:
        print('float')
        return float(text)

In [260]:
df_almost_total['price'] = df_almost_total['price'].apply(lambda x: process_price(x))

float


In [263]:
mask = df_almost_total['price'].isna()
to_fix = df_almost_total.loc[mask, :]

In [272]:
to_fix['title'].tolist()

['Belgian Waffles',
 'Orange Peel Cup',
 'Omelet In A Bag',
 'Instant Pot Braised Lamb With White Beans And Spinach ',
 "Mother's Day Breakfast Cobbler"]

In [ ]:
# fixed ingredients with annoying text

In [308]:
df_almost_total.to_csv("df_almost_ready_feb_6.csv", sep='|')

In [341]:
def to_check(text: str): 
    lower_text = text.lower()
    if lower_text.startswith('sure'): #'apologize' in lower_text: #or 'llm' in lower_text: or 'sure' in lower_text:
        return True
    else:
        return False

In [342]:

mask = df_almost_total['ingredients'].apply(lambda x: to_check(x))
sum(mask)

2

In [344]:
df_almost_total.loc[mask, "ingredients"].tolist()

['sure! here is a recipe for vegan protein muffins:\ningredients:\n- 1 cup plant-based milk (such as almond or soy milk)\n- 1 tablespoon apple cider vinegar\n- 1 ½ cups whole wheat flour\n- ½ cup vegan protein powder\n- ½ cup rolled oats\n- ¼ cup coconut sugar (or any other sweetener of choice)\n- 2 teaspoons baking powder\n- ½ teaspoon baking soda\n- ½ teaspoon salt\n- ½ teaspoon cinnamon\n- ¼ cup melted coconut oil\n- ½ cup unsweetened applesauce\n- 1 teaspoon vanilla extract\n- 1 cup chopped mixed nuts or seeds (such as almonds, walnuts, chia seeds, or sunflower seeds)\n- ½ cup dried fruits (such as raisins, cranberries, or chopped dates)',
 "sure! here's a recipe for veggie stir fry:\ningredients:\n- 1 tablespoon vegetable oil\n- 1 small onion, sliced\n- 2 cloves garlic, minced\n- 1 small carrot, sliced\n- 1 bell pepper, sliced\n- 1 cup broccoli florets\n- 1 cup sliced mushrooms\n- 1 cup snap peas\n- 1 cup tofu, cubed\n- 2 tablespoons low-sodium soy sauce\n- 1 tablespoon hoisin sau

In [347]:
ingredients_improved = []
for text in df_almost_total.loc[mask, "ingredients"].tolist():
    print(text)
    list_text = [txt for txt in text.split('\n') if len(txt) > 2]
    if len(list_text) == 1:
        new_text = list_text[0]
    else:
        new_text = '\n'.join(list_text[1:])
    print("****************************")
    print(new_text)
    ingredients_improved.append(new_text)
    print("----------------------------------------------")

sure! here is a recipe for vegan protein muffins:
ingredients:
- 1 cup plant-based milk (such as almond or soy milk)
- 1 tablespoon apple cider vinegar
- 1 ½ cups whole wheat flour
- ½ cup vegan protein powder
- ½ cup rolled oats
- ¼ cup coconut sugar (or any other sweetener of choice)
- 2 teaspoons baking powder
- ½ teaspoon baking soda
- ½ teaspoon salt
- ½ teaspoon cinnamon
- ¼ cup melted coconut oil
- ½ cup unsweetened applesauce
- 1 teaspoon vanilla extract
- 1 cup chopped mixed nuts or seeds (such as almonds, walnuts, chia seeds, or sunflower seeds)
- ½ cup dried fruits (such as raisins, cranberries, or chopped dates)
****************************
ingredients:
- 1 cup plant-based milk (such as almond or soy milk)
- 1 tablespoon apple cider vinegar
- 1 ½ cups whole wheat flour
- ½ cup vegan protein powder
- ½ cup rolled oats
- ¼ cup coconut sugar (or any other sweetener of choice)
- 2 teaspoons baking powder
- ½ teaspoon baking soda
- ½ teaspoon salt
- ½ teaspoon cinnamon
- ¼ cup m

In [348]:
len(ingredients_improved)

2

In [349]:
sum(mask)

2

In [350]:
df_almost_total.loc[mask, "ingredients"] = ingredients_improved

In [351]:
df_almost_total.to_csv("df_final_6_feb.csv", sep='|')

In [365]:
mask = df_almost_total['cultural_restriction'].isna()
print(sum(mask))
to_fix_carbs = df_almost_total.loc[mask, :]

1032


In [366]:
to_fix_carbs.to_csv("df_cultural_restriction_1032_fix.csv", sep="|")

In [2]:
df_almost_total = pd.read_csv("df_final_almost_ready.csv", 
                              sep="|", 
                              index_col=0)

In [3]:
df_almost_total.shape

(7227, 18)

In [123]:
df_prev = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/meals_collection/df_final_6_feb.csv",
                    sep='|', 
                    index_col=0)

In [124]:
list_prev = df_prev.loc[mask, 'title'].tolist()

In [126]:
df_almost_total.loc[mask, 'title'] = list_prev

In [4]:
n_mask = df_almost_total['title'].apply(lambda x: len(x) < 9)

In [5]:
sum(n_mask)

66

In [10]:
print(df_almost_total.loc[n_mask, 'title'].tolist())

['Vegan No', 'Oven', 'Quinoa', 'Falafel', 'Low', 'Greek', 'American', 'Hard', 'Edamame', 'Oat Bars', 'Muesli', 'Samosas', 'Frittata', 'Caprese', 'Lemon', 'Gazpacho', 'Veggie', 'Caponata', 'Indian', 'German', 'Pan', 'Energy', 'Turkish', 'Moroccan', 'Banana', 'Bacon', 'No', 'Pad Thai', 'Honey', 'Herb', 'Stir', 'Mexican', 'Israeli', 'Yogurt', 'Dairy', 'Gluten', 'Popcorn', 'Hummus', '3', 'Mujadara', 'Sushi', 'Halva', 'Ceviche', 'Borscht', 'Chili', 'Three', 'Italian', 'Muffins', 'Quiche', 'Fritters', 'Beignets', 'Porridge', 'Miso', 'One', 'Congee', 'Scones', 'Paratha', ' Arepas', ' Arepas', ' Cachapa', ' Ceviche', ' Ceviche', ' Humitas', ' Moqueca', 'Drommar', 'Monkfish']


In [7]:
new_list = ['Vegan No-Bake Energy Bites', 
            'Oven-Baked Zucchini Fries ', 
            'Quinoa-Stuffed Avocado', 
            'Falafel', 
            'Low-Calorie Veggie Stir-Fry', 
            'Greek-style baked beans',
            'American-style stacked pancakes', 
 'Edamame', 'Oat Bars', 'Muesli', 'Samosas', 'Frittata', 
 'Caprese-stuffed garlic bread', 
 'Lemon-garlic roasted chickpeas', 
 'Gazpacho', 
 'Veggie-Stuffed Pita Pockets', 
 'Caponata', 
 'Indian-Style Spinach and Chickpeas', 
 'German-style potato pancakes with applesauce', 
 'Pan-Fried Halloumi Salad', 
 'Energy-Boosting Smoothie', 
 'Turkish-style boiled eggs', 
 'Moroccan-style boiled eggs', 
 'Banana-nut muffins', 
 'Bacon-Wrapped Dates', 
 'No-Bake Energy Bars', 
 'Pad Thai', 
 'Honey-Glazed Baked Ham', 
 'Herb-roasted turkey breast with sweet potatoes', 
 'Stir-Fried Noodles with Vegetables', 
 'Mexican-style breakfast wrap with eggs and black beans', 
 'Israeli-style salad with scrambled eggs', 
 'Yogurt-covered pretzels', 
 'Dairy-Free Oatmeal Cookies', 
 'Gluten-Free Peanut Butter Cookies', 
 'Popcorn', 
 'Hummus-Stuffed Bell Pepper Cups', 
 'Bean Salad', 
 'Mujadara', 
 'Sushi', 
 'Halva', 
 'Ceviche', 
 'Borscht', 
 'Chili-lime grilled fish tacos', 
 'Three-bean chili with cornbread', 
 'Italian-style breakfast sandwich with prosciutto and mozzarella', 
 'Muffins', 
 'Quiche', 
 'Fritters', 
 'Beignets', 
 'Porridge', 
 'Miso-glazed salmon with jasmine rice and steamed vegetables', 
 'One-pot creamy chicken and mushroom pasta', 
 'Congee', 'Scones', 
 'Paratha', 
 'Arepas', 
 'Arepas', 
 'Cachapa',
 'Ceviche', 
 'Ceviche', 
 'Humitas', 
 'Moqueca', 
 'Drommar', 
 'Monkfish']

In [8]:
len(new_list)

65

In [5]:
df_almost_total.shape

(7227, 18)

In [102]:
df_almost_total.to_csv('df_almost_final_feb_7.csv', sep='|')

In [108]:
mask = df_almost_total['title'].apply(lambda x: len(x)<9)

In [109]:
sum(mask)

108

In [113]:
def fix_titles(raw_text:str):
    splitted = raw_text.split('\n')
    if len(splitted) > 2:
        return splitted[0]
    else:
        return ''

In [121]:
print(*df_almost_total.loc[mask, 'raw_text'].tolist())

65. Recipe: Vegan No-Bake Energy Bites
Calories: 200 per portion
Total Portions: 1
Allergic Warnings: None
Origin Country: Not applicable

 8. Oven-Baked Zucchini Fries 
   Calories per portion: 150
   Total portions: 2
   Allergic warnings: None
   Origin: N/A

 32. Quinoa-Stuffed Avocado 
    Calories per portion: 280
    Total portions: 2
    Allergic warnings: None
    Origin: N/A

 10. Falafel - 250 calories per portion, 4 portions, Gluten, Middle East

 3. Low-Calorie Veggie Stir-Fry (150 calories/portion, 3 portions) - China
 29. Greek-style baked beans: 300kcal per portion, 2 portions, no allergic warnings, origin: Greece
 38. American-style stacked pancakes: 400kcal per portion, 1 portion, contains gluten, origin: USA
 44. Edamame: 100 calories per portion, 1 portion, origin: Japan
    - Warning: None

 50. Oat Bars: 200 calories per portion, 1 portion, origin: -
    - Warning: Contains gluten

 53. Muesli: 180 calories per portion, 1 portion, origin: Switzerland
    - Warning

In [114]:
partial_list = df_almost_total.loc[mask, 'raw_text'].apply(lambda x: fix_titles(x))

In [116]:
new_mask = partial_list.apply(lambda x: len(x) > 0)

In [117]:
sum(new_mask)

73

In [118]:
partial_list[new_mask]

301                65. Recipe: Vegan No-Bake Energy Bites
588                         8. Oven-Baked Zucchini Fries 
612                           32. Quinoa-Stuffed Avocado 
1314    10. Falafel - 250 calories per portion, 4 port...
1899    44. Edamame: 100 calories per portion, 1 porti...
                              ...                        
7081    Quick-Fix Steak And Lemon-Pepper Potatoes1 cup...
7110    Izakaya-Style Okara, Vegetable, Mayonnaise and...
7121    Sugar-Pumpkin Puree1 sugar pumpkinSet a steame...
7126                      Monkfish2 lbs (.9 kg) monkfish 
7174    Korean-Style Steak and Lettuce Wraps1 lb flank...
Name: raw_text, Length: 73, dtype: object

In [ ]:
fix_list = ["Vegan No-Bake Energy Bites",
    "Oven-Baked Zucchini Fries",
    "Quinoa-Stuffed Avocado",
    "Falafel",
    "Low-Calorie Veggie Stir-Fry",
    "Greek-style baked beans",
    "American-style stacked pancakes",
    "Oat Bars",
    "Muesli",
    "Samosas", 
    'Frittata', 
    'Caprese-stuffed garlic bread',
    'Lemon-garlic roasted chickpeas',
    'Gazpacho',
    'Veggie-Stuffed Pita Pockets',
    'Caponata'
    
]

In [111]:
df_almost_total.loc[mask, 'raw_text'].tolist()

['Vegan No',
 'Oven',
 'Quinoa',
 'Falafel',
 'Low',
 'Greek',
 'American',
 'Edamame',
 'Oat Bars',
 'Muesli',
 'Samosas',
 'Frittata',
 'Caprese',
 'Lemon',
 'Gazpacho',
 'Veggie',
 'Caponata',
 'Indian',
 'German',
 'Pan',
 'Energy',
 'Turkish',
 'Moroccan',
 'Banana',
 'Bacon',
 'No',
 'Pad Thai',
 'Honey',
 'Herb',
 'Stir',
 'Mexican',
 'Israeli',
 'Yogurt',
 'Dairy',
 'Gluten',
 'Popcorn',
 'Hummus',
 '3',
 'Mujadara',
 'Sushi',
 'Halva',
 'Ceviche',
 'Borscht',
 'Chili',
 'Three',
 'Italian',
 'Muffins',
 'Quiche',
 'Fritters',
 'Beignets',
 'Porridge',
 'Miso',
 'One',
 'Congee',
 'Scones',
 'Paratha',
 ' Congee',
 ' Stir',
 ' Arepas',
 ' Arepas',
 ' Cachapa',
 ' Ceviche',
 ' Ceviche',
 ' Humitas',
 ' Moqueca',
 ' Stir',
 'Lentil',
 'Onion',
 'Oven',
 'Garlic',
 'Ginger',
 'Chic',
 'Drommar',
 'Chili',
 'Soup',
 'Just',
 'Almost',
 'Saffron',
 'Cola',
 'Basil',
 'Ginger',
 'Summer',
 'Turkey',
 'Lime',
 'Lemon',
 'Mini',
 'Elote',
 'Cocoa',
 'Orange',
 'Tahini',
 'Orange',
 'Ve

In [57]:
dff = pd.read_csv("/home/victor/Documents/Expectation_data_generation/df_ingredients_187_raw_new.csv",
                  sep='|',
                  index_col=0)

In [58]:
dff.shape

(185, 2)

In [66]:
# get ingredients 
def get_ingredients(text: str):
    answer = []
    values = text.split("\n\n")
    for val in values:
        if "Ingredients:" in val or '-' in val:
            answer.append(val)
    return answer

In [67]:
dff['list_ingredients'] = dff['raw_text'].apply(lambda x: get_ingredients(x))

In [92]:
mask = dff['list_ingredients'].apply(lambda x: len(x) > 0)
sum(mask)

183

In [94]:
dff.loc[mask, 'ingredients']=dff.loc[mask, 'list_ingredients'].apply(lambda x: x[0])

In [95]:
dff_to_fix = dff.loc[mask, :]

In [96]:
dff_to_fix.shape

(183, 4)

In [99]:
for idx, title in enumerate(dff_to_fix['title'].tolist()):
    new_mask = df_almost_total['title'] == title
    print(f"found: {sum(new_mask)}")
    if sum(new_mask) == 1:
        print("fixed")
        df_almost_total.loc[new_mask, 'ingredients'] = dff.loc[idx, 'ingredients']
    elif sum(new_mask) > 1:
         df_almost_total.loc[new_mask, 'ingredients'] = dff.loc[idx, 'ingredients']
        

found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 3
found: 1
fixed
found: 2
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 5
found: 4
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 2
found: 2
found: 1
fixed
found: 2
found: 1
fixed
found: 2
found: 1
fixed
found: 2
found: 1
fixed
found: 1
fixed
found: 2
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
fixed
found: 1
f

In [83]:
mask = df_almost_total['title'].str.contains('"Sun')

In [86]:
df_almost_total.loc[mask, 'title'] = '"Sun Dried" Tomatoes'

In [85]:
df_almost_total.loc[mask, 'raw_text']

6794    "Sun-Dried" Tomatoestomatoes \n  sea salt \n  ...
Name: raw_text, dtype: object

In [71]:
dff.loc[mask, 'title']

109    Hard
182    "Sun
Name: title, dtype: object

In [100]:
dff

,title,raw_text,list_ingredients,ingredients
0,Assorted cheese platter with crackers,"Sure, here's a basic list of ingredients with ...",[- 150g of cheddar cheese\n- 150g of brie chee...,- 150g of cheddar cheese\n- 150g of brie chees...
1,Mini Bagels with Cream Cheese,Sure! Here's the list of ingredients for 2 por...,[Ingredients:\n- 2 mini bagels\n- 4 tablespoon...,Ingredients:\n- 2 mini bagels\n- 4 tablespoons...
2,Miniature Bruschetta Pizzas,"Sure, here are the ingredients and quantities ...",[Ingredients:\n- 4 slices of baguette or Itali...,Ingredients:\n- 4 slices of baguette or Italia...
3,Mini Bagel Chips with Smoked Salmon,"Sure, here's a recipe for Mini Bagel Chips wit...",[Ingredients:\n- 2 mini bagels\n- 100g smoked ...,Ingredients:\n- 2 mini bagels\n- 100g smoked s...
4,Veggie Sticks with Hummus,"Sure, here's a recipe for Veggie Sticks with H...","[Ingredients:\n- 2 carrots, peeled and cut int...","Ingredients:\n- 2 carrots, peeled and cut into..."
...,...,...,...,...
180,Yuca con Chicharrón (El Salvador),Sure! Here are the ingredients with quantities...,"[- 1 lb yuca (cassava), peeled and cut into ch...","- 1 lb yuca (cassava), peeled and cut into chu..."
181,Johnny'S Navy Dressing,"Sure, here's a recipe for Johnny's Navy Dressi...",[Ingredients:\n- 2 tablespoons extra virgin ol...,Ingredients:\n- 2 tablespoons extra virgin oli...
182,"""Sun",I'd be happy to help with that! It seems like ...,[],NaN
183,Stuffed Baby Potatoes,"Sure, here's a recipe for Stuffed Baby Potatoe...",[Ingredients:\n- 10-12 baby potatoes\n- 1/4 cu...,Ingredients:\n- 10-12 baby potatoes\n- 1/4 cup...


In [6]:
def extract_nutritional_info(text: str, pattern: str):
    result = re.findall(pattern, text, re.IGNORECASE)
    if len(result) == 0:
        return np.nan
    else:
        return result[-1]

In [7]:
def extract_prep_time(text: str):
    lower_text = text.lower()
    res = re.findall(r'\d+\.?\d*', lower_text, re.IGNORECASE)
    if len(res) == 0:
        return np.nan
    else:
        val =  float(res[-1])
        if val < 3.0:
            return val*60.0
        else: 
            return val

In [8]:
grams_pattern = r'\d+\.?\d*(?=\s*g|\s*gram|\s*grams)'
partial_extractor = partial(extract_nutritional_info, pattern=grams_pattern)

In [9]:
dff['prep_time']=dff['raw_text'].apply(lambda x: extract_prep_time(x))

In [10]:
mask = dff['prep_time'].isna()
sum(mask)

0

In [28]:
ingredients_to_fix = dff
print(ingredients_to_fix.shape)

(1004, 2)


In [32]:
for idx, t in enumerate(ingredients_to_fix['title'].tolist()):
    mask = df_almost_total['title'] == t
    if df_almost_total.loc[mask, 'cultural_restriction'].isna().any():
        print(f"found: {sum(mask)}")
        len_mask = sum(mask)
        list_test = [ingredients_to_fix.loc[idx, 'raw_text']]
        if len_mask > 1:
            df_almost_total.loc[mask, 'cultural_restriction'] = list_test * len_mask
        elif len_mask == 1:
            df_almost_total.loc[mask, 'cultural_restriction'] = list_test[0]
        else: 
            pass

found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
found: 1
f

In [33]:
mask = df_almost_total['cultural_restriction'].isna()
sum(mask)

1

In [35]:
df_almost_total.loc[mask, 'cultural_restriction'] = 'chicken-based'

In [536]:
df_almost_total.loc[idx, "cooking_style"] = df_partial['cooking_style'].tolist()

In [537]:
mask = df_almost_total['cooking_style'].isna()
print(sum(mask))
to_f = df_almost_total.loc[mask, :]

0


In [22]:
mask = df_almost_total['prep_time'].isna()
print(sum(mask))

0


In [21]:
df_almost_total.loc[mask, 'prep_time'] = [15, 20]

In [24]:
df_almost_total['prep_time'] = pd.to_numeric(df_almost_total['prep_time'])

In [506]:
#df_almost_total.loc[mask, 'fiber'] = [5, 8, 2, 2, 6, 3, 1, 0]

In [539]:
#df_almost_total['protein']=pd.to_numeric(df_almost_total['protein'])
df_almost_total['cooking_style']=df_almost_total['cooking_style'].astype('string')

In [97]:
df_almost_total.to_csv("df_final_7_feb_titles_full.csv", sep="|")

In [37]:
def save_titles(title: str): 
    res = title
    if "-" in res:
        val = title.split('-')[0]
        if len(val) > 3:
            res = val
    if "**" in res:
        res = res.replace("**", "")
    return res

In [42]:
# process the titles 
df_almost_total['title'] = df_almost_total['title'].astype('string')

In [52]:
def check_ingredients(text: str):
    res = re.findall(r'\d+\.?\d*', text)
    if len(res) < 1:
        return True
    else:
        return False

In [53]:
mask = df_almost_total['ingredients'].apply(lambda x: check_ingredients(x))
print(sum(mask))

187


In [54]:
to_fix_ingredients = df_almost_total.loc[mask, :] 

In [56]:
to_fix_ingredients.to_csv('df_ingredients_187_to_fix.csv', sep="|")